Face Detection :

In [ ]:
!pip install retina-face opencv-python pillow matplotlib numpy facenet-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 941.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2

Link to Drive :

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Generating Comic story in the panel formats form a Math topic using API   :**

In [ ]:
import requests
import time

# Define API endpoint (using a smaller GPT-2 model that should work with free tier)
api_url = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.3"  # Changed to standard gpt2


# Define API key
api_key = "Api_link"  # Replace with your actual API key
# Function to interact with the API and generate a story

def generate_comic_story(topic):
    """Generate story for comics based on math topic and characters"""

    prompt=f""" Explain the math topic "{topic}" to 9th graders using a Marvel comic strip.Explains the topic step-by-step in a clear and simple way. Includes at least one example problem that is solved across multiple panels to teach the concept.

          Use 3 Marvel characters for single story.Each panel must have:
          - Panel [#]:
          - Scene Description
          - Characters
          - Actions
          - Dialogues: [Character]: [Dialogue]

          Make it fun, simple, educational, and humorous.
        """
    # prompt = """ Explain the math topic '{topic}' to 9th graders using a Marvel comic strip.
    #              Use one Marvel character per panel (you can use the same character in multiple panels or different characters).
    #              Each panel should be in the given format and should include:

    #       Panel [#]:
    #       Scene Description
    #       Character
    #       Action
    #       Dialogue: [Character]: [Single dialogue line]
    #       Make it fun, simple, educational, and humorous while ensuring each panel features exactly one character with one dialogue line.

    #     """
    # Request headers to include the API key
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    # Request body with improved parameters to avoid repetition
    data = {
        "inputs": prompt,
        "parameters": {
            "max_new_tokens": 1200,  # More controlled output length
            "temperature": 0.7,     # Slightly higher for more creativity
            "top_p": 0.92,          # Slightly higher for more variation
            "repetition_penalty": 1.2,  # Prevent repetition loops
            "do_sample": True       # Enable sampling for more varied outputs
        }
    }

    # Make the POST request to the API with retry mechanism
    max_retries = 3
    retry_delay = 2  # seconds

    for attempt in range(max_retries):
        try:
            response = requests.post(api_url, json=data, headers=headers, timeout=120)

            # Check if the request was successful
            if response.status_code == 200:
                result = response.json()

                # Handle different response formats
                if isinstance(result, list) and len(result) > 0:
                    return result[0].get('generated_text', "Story generation failed.")
                elif isinstance(result, dict):
                    return result.get('generated_text', str(result))
                else:
                    return str(result)

            # Handle specific error codes
            elif response.status_code == 503:
                print(f"Service unavailable (attempt {attempt+1}/{max_retries}), retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
                retry_delay *= 2  # Exponential backoff
                continue

            else:
                print(f"Error: {response.status_code} - {response.text}")
                return None

        except requests.exceptions.RequestException as e:
            print(f"Request error (attempt {attempt+1}/{max_retries}): {str(e)}")
            if attempt < max_retries - 1:
                time.sleep(retry_delay)
                retry_delay *= 2  # Exponential backoff
            else:
                return None

    return None  # If all retries failed

# Example usage - Get topic and characters from the user
topic = input("Enter the math topic: ")
# characters_input = input("Enter characters (comma separated): ")
# characters = [character.strip() for character in characters_input.split(",")] if characters_input.strip() else None

# # Set a default grade level or ask for it
# grade_level = input("Enter grade level (default is 'Grade 9'): ")
# if not grade_level:
#     grade_level = "Grade 9"

# Generate comic story
print("Generating story... (this might take a moment)")
comic_story = generate_comic_story(topic)

if comic_story:
    print("\n===== Generated Comic Story =====\n")
    print(comic_story)
else:
    print("\nFailed to generate the story. Please try again later.")

Enter the math topic: Linear Equations in Two Variables 
Generating story... (this might take a moment)

===== Generated Comic Story =====

 Explain the math topic "Linear Equations in Two Variables " to 9th graders using a Marvel comic strip.Explains the topic step-by-step in a clear and simple way. Includes at least one example problem that is solved across multiple panels to teach the concept.

          Use 3 Marvel characters for single story.Each panel must have:
          - Panel [#]:
          - Scene Description
          - Characters
          - Actions
          - Dialogues: [Character]: [Dialogue]

          Make it fun, simple, educational, and humorous.
        -----------------------------------------------------------------------------
        **Panel 1:**
        - Scene Description: Iron Man's lab filled with gadgets and computers.
        - Characters: Tony Stark (Iron Man), Spiderman, Hulk
        - Actions: Tony Stark is working on a new project; Spiderman swings i

In [ ]:
print(comic_story)

 Explain the math topic "Linear Equations in Two Variables " to 9th graders using a Marvel comic strip.Explains the topic step-by-step in a clear and simple way. Includes at least one example problem that is solved across multiple panels to teach the concept.

          Use 3 Marvel characters for single story.Each panel must have:
          - Panel [#]:
          - Scene Description
          - Characters
          - Actions
          - Dialogues: [Character]: [Dialogue]

          Make it fun, simple, educational, and humorous.
        -----------------------------------------------------------------------------
        **Panel 1:**
        - Scene Description: Iron Man's lab filled with gadgets and computers.
        - Characters: Tony Stark (Iron Man), Spiderman, Hulk
        - Actions: Tony Stark is working on a new project; Spiderman swings in looking curious while Hulk grunts quietly.
        - Dialogues:
            *Tony Stark*: Hey guys! Check this out – I just invented somet

Story 1 for demo purpose :

In [ ]:
comic_story="""
 Explain the math topic "Algebra" to 9th graders using a Marvel comic strip.Explains the topic step-by-step in a clear and simple way. Includes at least one example problem that is solved across multiple panels to teach the concept.

          Use 3 Marvel characters for single story. Use only one Marvel character per panel (you can use the same character in multiple panels or different characters). Each panel must have:
          - Panel [#]:
          - Scene Description
          - Characters
          - Actions
          - Dialogues: [Character]: [Dialogue]

          Make it fun, simple, educational, and humorous.
        --------------------------------------------------
        **Panel 1:**
        - Scene Description: The Avengers' base. Black Widow is explaining Algebra to Captain America.
        - Characters: Black Widow, Captain America
        - Actions: Black Widow writes on a whiteboard, Cap listens attentively.
        - Dialogues:
            Black Widow: Hey Cap, ever wondered what Algebra means? It's like a secret code we superheroes sometimes need to crack!
            Captain America: Secret codes, huh? I thought you just used your spy gadgets...
            Black Widow: Well, those help too but let me show you how algebra works with this equation – x + y = z.

        **Panel 2:**
        - Scene Description: Black Widow demonstrates the meaning of variables 'x', 'y' and 'z'.
        - Characters: Black Widow, Captain America
        - Actions: Black Widow draws symbols on the whiteboard representing 'x','y' and 'z'.
        - Dialogues:
            Black Widow: See these letters up here, x, y, and z? They represent unknown numbers. We solve equations by finding their values.
            Captain America: Unknown numbers, got it. Like when we don't know how many enemies Loki will send next time?
            Black Widow: Exactly, Cap! Now watch as I work out an equation together. Let's say if I add 5 apples (represented by x) and 3 bananas (represented by y), they make a total of 8 fruits (or z). So our equation becomes: x + y = z

        **Panel 3:**
        - Scene Description: Black Widow solves the equation, helping Captain America understand.
        - Characters: Black Widow, Captain America
        - Actions: Black Widow uses erasable markers to write and change numbers on the board.
        - Dialogues:
            Black Widow: If we want to find the value of x (the number of apples), we can set up another equation: z - y = x. Subtracting y from both sides gives us x equals z minus y.
            Captain America: Wait, so if there are 8 fruits altogether and I subtract the 3 bananas, I get the number of apples left?
            Black Widow: That's right, Cap! Using this logic, since z is 8 and y is 3, then x would be 8 - 3 which equals 5.

        **Panel 4:**
        - Scene Description: Captain America tries solving an equation himself.
        - Characters: Captain America, Iron Man
        - Actions: Captain America writes an equation on a piece of paper while Iron Man looks over his shoulder.
        - Dialogues:
            Captain America: You know what, Widow? I think I've got the hang of this algebra thing. How about I try my hand at some equations now?
            Iron Man: Great! Just remember to follow the steps carefully. Let's see… What do you have there?
            Captain America: Hmmm… Here goes: If I eat 2 slices of pizza (x) and drink 3 glasses of milk (y), will it make me fuller than eating 4 pieces of cake (z)? So my question is: Is x + y > z ?
            Iron Man: Not exactly an equation, but close enough for heroic purposes, Cap! Keep practicing and soon you'll master algebra too!
        """

In [ ]:
comic_story="""
 Explain the math topic "Linear Equations in Two Variables " to 9th graders using a Marvel comic strip.Explains the topic step-by-step in a clear and simple way. Includes at least one example problem that is solved across multiple panels to teach the concept.

          Use 3 Marvel characters for single story.Each panel must have:
          - Panel [#]:
          - Scene Description
          - Characters
          - Actions
          - Dialogues: [Character]: [Dialogue]

          Make it fun, simple, educational, and humorous.
        -----------------------------------------------------------------------------
        **Panel 1:**
        - Scene Description: Iron Man's lab filled with gadgets and computers.
        - Characters: Tony Stark (Iron Man), Spiderman, Hulk
        - Actions: Tony Stark is working on a new project; Spiderman swings in looking curious while Hulk grunts quietly.
        - Dialogues:
            *Tony Stark*: Hey guys! Check this out – I just invented something amazing! It can solve any linear equation in two variables!
            *Spiderman*: Whoa, cool! How does it work?
            *Hulk*: Me smash things... not equations.
            *Tony Stark*: Well, let me explain!

        **Panel 2:**
        - Scene Description: Tony explains his invention.
        - Characters: Tony Stark, Spiderman, Hulk
        - Actions: Tony points towards a large whiteboard showing an equation: y = mx + b.
        - Dialogues:
            *Tony Stark*: Imagine we have a graph of dots representing different pairs of x and y values. We want to find a straight line passing through those dots.
            *Spiderman*: So, we draw a best fit line right?
            *Tony Stark*: Exactly! And here comes our hero - Linear Equation.

        **Panel 3:**
        - Scene Description: Graphing the equation on the board.
        - Characters: Tony Stark, Spiderman, Hulk
        - Actions: Tony draws the graph passing through three given points (x₁,y₁);(x₂,y₂) & (x₃,y₃).
        - Dialogues:
            *Tony Stark*: Let’s pick three points from our data set. Now, if we connect these points with a line, we get our equation.
            *Spiderman*: But how do we find m and b? The slope and y-intercept, you know?
            *Tony Stark*: Great question! Let's dive into that next time. For now, just remember that every straight line has its unique pair of m (slope) and b (y-intercept).

        **Panel 4:**
        - Scene Description: Tony assigns them homework to practice solving linear equations.
        - Characters: Tony Stark, Spiderman, Hulk
        - Actions: Tony hands over sheets containing various problems.
        - Dialogues:
            *Tony Stark*: Alright folks, it's home time. Here are some practice problems for you. Solve them, so we can continue discussing more advanced stuff soon!
            *Spiderman*: Awesome! Thanks, Tony!
            *Hulk*: Smash! Learn math!

"""

Save the comic story in a file names : comic_story.txt :

In [ ]:
def save_to_comic_story(data):
    """Save variable data to comic_story.txt file"""
    with open("comic_story.txt", "w") as file:
        file.write(data)
    print(f"Data successfully saved to comic_story.txt")

save_to_comic_story(comic_story)

Data successfully saved to comic_story.txt


Seperating every dialogue to a seperate panel :

In [ ]:
import re

def format_comic_dialogue_individual_panels(comic_story):
    """
    Format a comic story into individual panels per dialogue line.
    Enhanced to handle various input formats for comic scripts including asterisk-wrapped character names.
    Each dialogue line becomes its own panel with preserved metadata.

    Returns a formatted string with panels and dialogues.
    """
    formatted_output = ""
    panel_counter = 1

    # Try different panel pattern formats
    panel_patterns = [
        r'\*\*Panel\s+\d+:?\*\*',  # **Panel 1:** or **Panel 1**
        r'Panel\s+\d+:',            # Panel 1:
        r'PANEL\s+\d+:?',           # PANEL 1: or PANEL 1
        r'P\d+:?'                   # P1: or P1
    ]

    # Find the pattern that works best
    best_pattern = None
    max_panels = 0

    for pattern in panel_patterns:
        test_panels = re.split(pattern, comic_story)
        if len(test_panels) > max_panels:
            max_panels = len(test_panels)
            best_pattern = pattern

    # Use the best pattern to split
    if best_pattern:
        panels = re.split(best_pattern, comic_story)
    else:
        panels = [comic_story]

    # Remove content before first actual panel if it doesn't contain useful info
    if len(panels) > 1 and not any(keyword in panels[0].lower() for keyword in ['character', 'scene', 'action', 'dialogue']):
        panels = panels[1:]

    for panel in panels:
        if not panel.strip():
            continue

        # Initialize panel metadata
        scene_desc = ""
        chars = ""
        actions = ""

        # Extract scene description
        scene_patterns = [
            r'- Scene Description:(.*?)(?=-\s+Characters|-\s+Actions|-\s+Dialogues?|$)',
            r'Scene Description:(.*?)(?=Character|Action|Dialogue|$)',
            r'Scene:(.*?)(?=Character|Action|Dialogue|$)',
            r'Setting:(.*?)(?=Character|Action|Dialogue|$)'
        ]

        for pattern in scene_patterns:
            scene_match = re.search(pattern, panel, re.DOTALL | re.IGNORECASE)
            if scene_match:
                scene_desc = scene_match.group(1).strip()
                break

        # Extract characters
        char_patterns = [
            r'- Characters:(.*?)(?=-\s+Actions|-\s+Dialogues?|$)',
            r'Characters:(.*?)(?=Action|Dialogue|$)',
            r'Character\(s\):(.*?)(?=Action|Dialogue|$)'
        ]

        for pattern in char_patterns:
            chars_match = re.search(pattern, panel, re.DOTALL | re.IGNORECASE)
            if chars_match:
                chars = chars_match.group(1).strip()
                break

        # Extract actions
        action_patterns = [
            r'- Actions:(.*?)(?=-\s+Dialogues?|$)',
            r'Actions:(.*?)(?=Dialogue|$)',
            r'Action:(.*?)(?=Dialogue|$)'
        ]

        for pattern in action_patterns:
            actions_match = re.search(pattern, panel, re.DOTALL | re.IGNORECASE)
            if actions_match:
                actions = actions_match.group(1).strip()
                break

        # Prepare metadata
        metadata_lines = []
        if scene_desc:
            metadata_lines.append(f"Scene Description: {scene_desc}")
        if chars:
            metadata_lines.append(f"Characters: {chars}")
        if actions:
            metadata_lines.append(f"Actions: {actions}")

        metadata = "\n".join(metadata_lines)

        # Extract dialogues with broader patterns
        dialogue_section = ""

        # First try structured dialogue sections
        dialogue_patterns = [
            r'- Dialogues?:(.*?)(?=\*\*Panel|\Z)',
            r'Dialogues?:(.*?)(?=Panel|\Z)',
            r'Dialog(?:ue)?:(.*?)(?=Panel|\Z)'
        ]

        for pattern in dialogue_patterns:
            dialogue_match = re.search(pattern, panel, re.DOTALL | re.IGNORECASE)
            if dialogue_match:
                dialogue_section = dialogue_match.group(1).strip()
                break

        # If no structured dialogue section found, try to extract dialogues from the whole panel
        if not dialogue_section:
            dialogue_section = panel

        # Fix the indentation issue - remove leading spaces from dialogue lines
        dialogue_section = re.sub(r'\n\s+', '\n', dialogue_section)

        # Enhanced dialogue patterns to handle multiple formats:
        # 1. Dash format: "- CHARACTER: dialogue"
        # 2. Asterisk format: "*Character Name*: dialogue"
        # 3. Regular format: "Character Name: dialogue"
        # 4. All caps format: "CHARACTER: dialogue"
        dialogue_patterns_enhanced = [
            # Dash format with optional asterisks
            r'-\s*\*?([A-Za-z][A-Za-z\s\.\-\']+)\*?:\s*(.*?)(?=\n\s*-\s*\*?[A-Za-z]|\n\s*\*[A-Za-z]|\n[A-Za-z][A-Za-z\s\.\-\']+:|$)',
            # Asterisk-wrapped character names
            r'\*([A-Za-z][A-Za-z\s\.\-\']+)\*:\s*(.*?)(?=\n\s*\*[A-Za-z][A-Za-z\s\.\-\']+\*:|\n[A-Za-z][A-Za-z\s\.\-\']+:|$)',
            # Regular character names (including all caps)
            r'([A-Za-z][A-Za-z\s\.\-\']*[A-Za-z]):\s*(.*?)(?=\n[A-Za-z][A-Za-z\s\.\-\']*[A-Za-z]:|$)'
        ]

        dialogues = []
        for dialogue_pattern in dialogue_patterns_enhanced:
            found_dialogues = re.findall(dialogue_pattern, dialogue_section, re.DOTALL)
            if found_dialogues:
                dialogues = found_dialogues
                break

        # If no dialogues found with the patterns, try to extract anything that looks like dialogue
        if not dialogues:
            # Look for any text that might be dialogue in quotes
            simple_dialogue = re.findall(r'"([^"]+)"', dialogue_section)
            if simple_dialogue:
                # Assume NARRATOR for simple dialogues if no character specified
                dialogues = [("NARRATOR", line) for line in simple_dialogue]

        # Skip panels with no valid dialogues
        if not dialogues:
            continue

        # Create a separate panel for each dialogue line
        for character, text in dialogues:
            cleaned_text = re.sub(r'\s+', ' ', text).strip()
            cleaned_char = re.sub(r'\s+', ' ', character).strip().upper()

            if not cleaned_char or not cleaned_text:
                continue

            # Skip empty or overly short texts
            if len(cleaned_text) < 2:
                continue

            # Remove any extra formatting characters
            cleaned_text = re.sub(r'^\s*[-\*]*\s*', '', cleaned_text)
            cleaned_text = re.sub(r'\s*[-\*]*\s*$', '', cleaned_text)

            formatted_output += f"Panel {panel_counter}:\n"
            if metadata:
                formatted_output += metadata + "\n\n"
            formatted_output += f"{cleaned_char}: {cleaned_text}\n"
            formatted_output += "-" * 50 + "\n"
            panel_counter += 1

    return formatted_output.strip()

# Testing function for the new Linear Equations format
def test_linear_equations_comic():
    """Test function specifically for the Linear Equations comic format with dash dialogues"""
    linear_equations_comic = """Panel 1:
- Scene Description: Tony Stark's Workshop (Stark Tower)
- Characters: Iron Man, Captain America, Black Widow
- Actions: The heroes are gathered around a table with various gadgets and equations written on a board.
- Dialogues:
    - IRON MAN: Alright team, today we're gonna learn about something super important – Linear Equations in Two Variables!
    - CAPTAIN AMERICA: Sounds complicated, Stark... But I'm all ears.
    - BLACK WIDOW: Me too. We need this if we want to crack some villain's secret hideout plans.

Panel 2:
- Scene Description: Whiteboard with an equation: y = mx + b
- Characters: Iron Man, Captain America, Black Widow
- Actions: Iron Man points towards the whiteboard explaining the parts of the equation.
- Dialogues:
    - IRON MAN: This right here, my friends, is our new best friend – the standard form of a linear equation in two variables. It has three main components: m, x, and b.
    - CAPTAIN AMERICA: So, what does each part mean?
    - BLACK WIDOW: I think it's like telling us how much y changes based on how much x increases or decreases, multiplied by some number 'm.' And there's also a constant value 'b,' which represents where the line starts.
    - IRON MAN: Exactly! Now let me show you guys how to find the values of 'm' and 'b' from a given set of points.

Panel 3:
- Scene Description: Table with a grid paper and markings showing two points (A(x₁,y₁), B(x₂,y₂)).
- Characters: Iron Man, Captain America, Black Widow
- Actions: Iron Man draws lines connecting points A and B and then adds more points along those lines. He then shows how to calculate the slope 'm' and the y-intercept 'b'.
- Dialogues:
    - IRON MAN: Let's say we have these two points, point A (x₁, y₁) and point B (x₂, y₂). To find the slope 'm', first subtract x₁ from x₂, divide the result by y₂ minus y₁.
    - CAPTAIN AMERICA: Got it. So, if point A is (-2,4) and point B is (5,7), we would do: m=[7-(4)]/[5--(-2)]=3/7.
    - BLACK WIDOW: Nice work, Cap. Next up, finding the y-intercept 'b'. That tells us where the line crosses the y-axis when x equals zero.
    - IRON MAN: Right! For our case, if m=3/7, then substitute x with 0 in the equation: b=y−mx=(y0)-(3/7)(0)=y0.

Panel 4:
- Scene Description: Table with completed graph representing the linear equation y = 3/7x + 4
- Characters: Iron Man, Captain America, Black Widow
- Actions: Iron Man sketches out the graph of the linear equation they found earlier.
- Dialogues:
    - IRON MAN: Here's the final piece – drawing the actual graph of our newly discovered linear equation.
    - CAPTAIN AMERICA: Wow, now that looks familiar! Is this the same pattern as HYDRA's secret base layout I saw during our last mission?
    - BLACK WIDOW: Looks like it, Cap. Thanks to Linear Equations in Two Variables, we might be able to predict their next move!

Panel 5:
- Scene Description: Heroes high-fiving and looking determined
- Characters: Iron Man, Captain America, Black Widow
- Actions: Team celebrates their successful understanding of the lesson while making future plans.
- Dialogues:
    - IRON MAN: Great job, everyone! With Linear Equations in Two Variables under our belts, we'll be unstoppable against any threats coming our way!
    - CAPTAIN AMERICA: Agreed! Time to put our new skills to good use and save the world once again!
    - BLACK WIDOW: Bring it on! When we combine forces, nothing can stand in our way!"""

    return linear_equations_comic

# Debug function to help diagnose issues
def debug_comic_parsing(comic_story):
    """Helper function to debug the parsing of a comic story"""
    print("==== DEBUG INFO ====")

    # Test panel splitting
    panel_patterns = [
        r'\*\*Panel\s+\d+:?\*\*',  # **Panel 1:** or **Panel 1**
        r'Panel\s+\d+:',            # Panel 1:
        r'PANEL\s+\d+:?',           # PANEL 1: or PANEL 1
        r'P\d+:?'                   # P1: or P1
    ]

    for pattern in panel_patterns:
        panels = re.split(pattern, comic_story)
        print(f"Pattern '{pattern}' produces {len(panels)} panels")

    # Use the first working pattern for further debugging
    pattern = None
    panels = []
    for test_pattern in panel_patterns:
        test_panels = re.split(test_pattern, comic_story)
        if len(test_panels) > 1:
            pattern = test_pattern
            panels = test_panels
            break

    if panels and len(panels) > 1:
        # Show first panel content
        print(f"\nUsing pattern: {pattern}")
        print("First panel content (truncated):")
        sample_panel = panels[1][:400] + "..." if len(panels[1]) > 400 else panels[1]
        print(sample_panel)

        # Test dialogue extraction
        print("\nDialogue extraction test:")
        dialogue_section = sample_panel

        # Try to find dialogues section
        dialogue_match = re.search(r'- Dialogues?:(.*?)(?=Panel|\Z)', sample_panel, re.DOTALL)
        if dialogue_match:
            dialogue_section = dialogue_match.group(1).strip()
            print("Found dialogue section!")
        else:
            print("No specific dialogue section found, using whole panel")

        # Remove indentation that might be causing issues
        dialogue_section = re.sub(r'\n\s+', '\n', dialogue_section)

        # Test all dialogue patterns
        patterns_to_test = [
            r'-\s*\*?([A-Za-z][A-Za-z\s\.\-\']+)\*?:\s*(.*?)(?=\n\s*-\s*\*?[A-Za-z]|\n\s*\*[A-Za-z]|\n[A-Za-z][A-Za-z\s\.\-\']+:|$)',
            r'\*([A-Za-z][A-Za-z\s\.\-\']+)\*:\s*(.*?)(?=\n\s*\*[A-Za-z][A-Za-z\s\.\-\']+\*:|\n[A-Za-z][A-Za-z\s\.\-\']+:|$)',
            r'([A-Za-z][A-Za-z\s\.\-\']*[A-Za-z]):\s*(.*?)(?=\n[A-Za-z][A-Za-z\s\.\-\']*[A-Za-z]:|$)'
        ]

        for i, dialogue_pattern in enumerate(patterns_to_test):
            dialogues = re.findall(dialogue_pattern, dialogue_section, re.DOTALL)
            print(f"Pattern {i+1} found {len(dialogues)} dialogues")
            for j, (char, text) in enumerate(dialogues[:3]):  # Show first three dialogues
                preview_text = text[:50] + "..." if len(text) > 50 else text
                print(f"  {j+1}. {char}: {preview_text}")

    print("==== END DEBUG ====")

# Main execution
if __name__ == "__main__":
    # Read comic story from file
    input_file = "/content/comic_story.txt"

    try:
        with open(input_file, "r", encoding="utf-8") as f:
            comic_story = f.read()
        print(f"✅ Successfully read comic story from '{input_file}'")
        print(f"📄 File size: {len(comic_story)} characters")

    except FileNotFoundError:
        print(f"❌ Error: File '{input_file}' not found!")
        print("🔄 Falling back to test comic for demonstration...")
        comic_story = test_linear_equations_comic()

    except Exception as e:
        print(f"❌ Error reading file '{input_file}': {e}")
        print("🔄 Falling back to test comic for demonstration...")
        comic_story = test_linear_equations_comic()

    print("\n🔍 Testing Comic Format:")
    debug_comic_parsing(comic_story)

    print("\n" + "="*60)
    print("📊 FORMATTED OUTPUT (Each dialogue becomes its own panel):")
    print("="*60)

    formatted_output = format_comic_dialogue_individual_panels(comic_story)
    print(formatted_output)

    # Save to file
    output_file = "comic_output.txt"
    try:
        with open(output_file, "w", encoding="utf-8") as f:
            f.write(formatted_output)
        print(f"\n✅ Output successfully written to '{output_file}' with {len(formatted_output)} characters.")

        # Count how many individual panels were created
        panel_count = formatted_output.count("Panel ")
        print(f"📊 Total individual dialogue panels created: {panel_count}")

    except Exception as e:
        print(f"❌ Error writing to file: {e}")

✅ Successfully read comic story from '/content/comic_story.txt'
📄 File size: 3711 characters

🔍 Testing Comic Format:
==== DEBUG INFO ====
Pattern '\*\*Panel\s+\d+:?\*\*' produces 5 panels
Pattern 'Panel\s+\d+:' produces 5 panels
Pattern 'PANEL\s+\d+:?' produces 1 panels
Pattern 'P\d+:?' produces 1 panels

Using pattern: \*\*Panel\s+\d+:?\*\*
First panel content (truncated):

        - Scene Description: The Avengers' base. Black Widow is explaining Algebra to Captain America.
        - Characters: Black Widow, Captain America
        - Actions: Black Widow writes on a whiteboard, Cap listens attentively.
        - Dialogues:
            Black Widow: Hey Cap, ever wondered what Algebra means? It's like a secret code we superheroes sometimes need to crack!
            ...

Dialogue extraction test:
Found dialogue section!
Pattern 1 found 0 dialogues
Pattern 2 found 0 dialogues
Pattern 3 found 1 dialogues
  1. Black Widow: Hey Cap, ever wondered what Algebra means? It's li...
==== END DE

Dataset for character descriptions :

In [ ]:
import pandas as pd
import json

# Create a comprehensive Marvel characters dataset
marvel_characters = [
    {
        "id": 1,
        "name": "Iron Man",
        "real_name": "Tony Stark",
        "powers": ["Genius Intellect", "Engineering", "Powered Armor"],
        "description": "Billionaire industrialist and genius inventor who created a powered suit of armor to save his life and escape captivity. Now uses his suit and technological devices to protect the world as Iron Man.",
        "visual_description": "Tall man in sleek metallic armor with glowing blue arc reactor in chest, hovering with repulsors in palms, angular helmet with glowing white eyes",
        "color_scheme": ["Red", "Gold", "Blue (arc reactor)", "White (repulsors)"],
        "distinctive_features": ["Glowing blue arc reactor", "Metallic armor plating", "Palm repulsors", "Angular faceplate"],
        "primary_colors": "Red and Gold",
        "first_appearance": "Tales of Suspense #39 (1963)",
        "team_affiliations": ["Avengers", "Illuminati"],
        "origin": "Earth-616",
        "height_cm": 185,
        "weight_kg": 102,
        "is_villain": False
    },
    {
        "id": 2,
        "name": "Captain America",
        "real_name": "Steve Rogers",
        "powers": ["Enhanced Strength", "Enhanced Durability", "Enhanced Agility", "Healing Factor", "Expert Tactician"],
        "description": "A frail young man enhanced to the peak of human physical potential by an experimental serum. Wields a nearly indestructible shield made of vibranium. Embodies the ideals of liberty and justice.",
        "visual_description": "Muscular man in blue suit with large white star on chest, red and white horizontal stripes on torso, winged helmet, holding circular shield with concentric red, white, and blue rings",
        "color_scheme": ["Blue", "Red", "White", "Silver (shield)"],
        "distinctive_features": ["White star on chest", "Striped pattern", "Circular shield", "Winged helmet", "American flag motif"],
        "primary_colors": "Red, White, and Blue",
        "first_appearance": "Captain America Comics #1 (1941)",
        "team_affiliations": ["Avengers", "Howling Commandos", "S.H.I.E.L.D."],
        "origin": "Earth-616",
        "height_cm": 188,
        "weight_kg": 108,
        "is_villain": False
    },
    {
        "id": 3,
        "name": "Thor",
        "real_name": "Thor Odinson",
        "powers": ["Superhuman Strength", "Weather Manipulation", "Flight", "Longevity", "Dimensional Travel"],
        "description": "Asgardian god of thunder with control over lightning and storms. Wields the enchanted hammer Mjolnir, which grants him flight and other powers. Member of both the Avengers and the royal family of Asgard.",
        "visual_description": "Tall muscular man with long flowing blonde hair, red cape, silver chainmail armor with circular chest disc, carrying ornate hammer with short handle, blue and yellow lightning crackling around him",
        "color_scheme": ["Red", "Silver", "Blue", "Yellow", "Blonde"],
        "distinctive_features": ["Long blonde hair", "Red cape", "Mjolnir hammer", "Circular chest armor", "Lightning effects"],
        "primary_colors": "Red and Silver",
        "first_appearance": "Journey into Mystery #83 (1962)",
        "team_affiliations": ["Avengers", "Asgardian Royal Family"],
        "origin": "Asgard",
        "height_cm": 198,
        "weight_kg": 290,
        "is_villain": False
    },
    {
        "id": 4,
        "name": "Hulk",
        "real_name": "Bruce Banner",
        "powers": ["Superhuman Strength", "Superhuman Durability", "Regeneration", "Anger Empowerment"],
        "description": "A brilliant scientist transformed into a giant green monster when angry or stressed. The angrier he gets, the stronger he becomes. Often struggles with controlling his transformations.",
        "visual_description": "Massive muscular giant with bright green skin, bulging veins, wild dark hair, wearing torn purple pants, clenched fists, mid-roar expression",
        "color_scheme": ["Green", "Purple", "Dark Brown (hair)"],
        "distinctive_features": ["Bright green skin", "Massive muscular build", "Torn purple pants", "Protruding brow", "Bulging veins"],
        "primary_colors": "Green and Purple",
        "first_appearance": "The Incredible Hulk #1 (1962)",
        "team_affiliations": ["Avengers", "Defenders", "Illuminati"],
        "origin": "Earth-616",
        "height_cm": 244,
        "weight_kg": 635,
        "is_villain": False
    },
    {
        "id": 5,
        "name": "Black Widow",
        "real_name": "Natasha Romanoff",
        "powers": ["Expert Spy", "Master Martial Artist", "Enhanced Immune System", "Expert Tactician"],
        "description": "Former Russian spy trained as an assassin from childhood. Later defected to S.H.I.E.L.D. and became an Avenger. Exceptional at espionage, infiltration, and combat.",
        "visual_description": "Athletic woman with shoulder-length curly red hair, sleek black leather bodysuit, silver utility belt, wrist gauntlets with red hourglass symbol, poised in combat stance",
        "color_scheme": ["Black", "Red", "Silver"],
        "distinctive_features": ["Curly red hair", "Black leather suit", "Red hourglass symbol", "Wrist gauntlets", "Athletic build"],
        "primary_colors": "Black and Red",
        "first_appearance": "Tales of Suspense #52 (1964)",
        "team_affiliations": ["Avengers", "S.H.I.E.L.D.", "Champions"],
        "origin": "Earth-616",
        "height_cm": 170,
        "weight_kg": 59,
        "is_villain": False
    },
    {
        "id": 6,
        "name": "Spider-Man",
        "real_name": "Peter Parker",
        "powers": ["Superhuman Strength", "Wall-Crawling", "Spider-Sense", "Enhanced Agility", "Web-Shooters"],
        "description": "Bitten by a radioactive spider, gained the proportionate strength and agility of an arachnid. Created web-shooters to complement his powers. Lives by the motto 'With great power comes great responsibility.'",
        "visual_description": "Lean athletic figure in bright red and blue suit with black web pattern throughout, large white triangular eye lenses, spider emblem on chest, web-shooters on wrists",
        "color_scheme": ["Red", "Blue", "Black", "White"],
        "distinctive_features": ["Web pattern design", "Large white eye lenses", "Spider emblem", "Red and blue color split", "Lean muscular build"],
        "primary_colors": "Red and Blue",
        "first_appearance": "Amazing Fantasy #15 (1962)",
        "team_affiliations": ["Avengers", "Future Foundation", "New Avengers"],
        "origin": "Earth-616",
        "height_cm": 178,
        "weight_kg": 76,
        "is_villain": False
    },
    {
        "id": 7,
        "name": "Doctor Strange",
        "real_name": "Stephen Strange",
        "powers": ["Mystic Arts", "Spellcasting", "Astral Projection", "Dimensional Travel", "Teleportation"],
        "description": "Former neurosurgeon who lost the use of his hands. Became the Sorcerer Supreme, master of the mystic arts and primary defender of Earth against magical and mystical threats.",
        "visual_description": "Distinguished man with graying temples and goatee, blue tunic with high collar, red levitating Cloak of Levitation, golden Eye of Agamotto amulet, glowing orange magical circles around hands",
        "color_scheme": ["Blue", "Red", "Gold", "Orange", "Gray"],
        "distinctive_features": ["Red levitating cloak", "Eye of Agamotto amulet", "Mystical hand gestures", "Goatee and graying temples", "Magical energy effects"],
        "primary_colors": "Blue and Red",
        "first_appearance": "Strange Tales #110 (1963)",
        "team_affiliations": ["Avengers", "Defenders", "Illuminati"],
        "origin": "Earth-616",
        "height_cm": 188,
        "weight_kg": 81,
        "is_villain": False
    },
    {
        "id": 8,
        "name": "Black Panther",
        "real_name": "T'Challa",
        "powers": ["Enhanced Strength", "Enhanced Speed", "Enhanced Durability", "Enhanced Senses", "Vibranium Suit"],
        "description": "King of the technologically advanced African nation of Wakanda. Gains enhanced abilities from consuming the Heart-Shaped Herb. Wears a vibranium-woven suit that absorbs kinetic energy.",
        "visual_description": "Athletic man in sleek black vibranium suit with subtle geometric patterns, panther-head helmet with pointed ears, glowing white/silver eyes, silver claw accents, retractable claws",
        "color_scheme": ["Black", "Silver", "White", "Purple (energy)"],
        "distinctive_features": ["Panther helmet with ears", "All-black suit", "Glowing white eyes", "Silver claw marks", "Geometric patterns"],
        "primary_colors": "Black and Silver",
        "first_appearance": "Fantastic Four #52 (1966)",
        "team_affiliations": ["Avengers", "Illuminati"],
        "origin": "Earth-616",
        "height_cm": 183,
        "weight_kg": 91,
        "is_villain": False
    },
    {
        "id": 9,
        "name": "Captain Marvel",
        "real_name": "Carol Danvers",
        "powers": ["Superhuman Strength", "Energy Absorption", "Energy Projection", "Flight", "Near-Invulnerability"],
        "description": "Former Air Force pilot who gained superhuman powers when exposed to alien technology. Can absorb and redirect energy, granting her tremendous strength, durability, and the ability to fly in space.",
        "visual_description": "Confident woman with short blonde hair, red bodysuit with blue sections, gold accents and belt, star emblem on chest, glowing golden fists with cosmic energy aura around body",
        "color_scheme": ["Red", "Blue", "Gold", "Blonde"],
        "distinctive_features": ["Star emblem", "Short blonde hair", "Glowing fists", "Cosmic energy aura", "Military-style suit design"],
        "primary_colors": "Red, Blue, and Gold",
        "first_appearance": "Marvel Super-Heroes #13 (1968)",
        "team_affiliations": ["Avengers", "Alpha Flight", "Starforce"],
        "origin": "Earth-616",
        "height_cm": 180,
        "weight_kg": 74,
        "is_villain": False
    },
    {
        "id": 10,
        "name": "Scarlet Witch",
        "real_name": "Wanda Maximoff",
        "powers": ["Reality Manipulation", "Chaos Magic", "Energy Projection", "Telekinesis", "Mind Control"],
        "description": "Powerful sorceress capable of manipulating probability and reality itself. Her abilities are tied to chaos magic. Often struggles with controlling the full extent of her powers.",
        "visual_description": "Woman with long dark hair, red leather corset and coat, distinctive red headpiece with points, hands and eyes glowing with swirling crimson energy tendrils",
        "color_scheme": ["Red", "Dark Red", "Crimson", "Black"],
        "distinctive_features": ["Pointed red headpiece", "Red leather outfit", "Crimson energy effects", "Long dark hair", "Glowing red eyes when using powers"],
        "primary_colors": "Red and Crimson",
        "first_appearance": "X-Men #4 (1964)",
        "team_affiliations": ["Avengers", "Brotherhood of Mutants"],
        "origin": "Earth-616",
        "height_cm": 170,
        "weight_kg": 59,
        "is_villain": False
    },
    {
        "id": 11,
        "name": "Thanos",
        "real_name": "Thanos",
        "powers": ["Superhuman Strength", "Superhuman Durability", "Immortality", "Energy Manipulation", "Telekinesis"],
        "description": "Powerful warlord from the moon Titan. Obsessed with death and acquiring ultimate power. Most known for his quest to collect the Infinity Stones to wipe out half of all life in the universe.",
        "visual_description": "Massive humanoid with deep purple skin, pronounced chin with vertical ridges, bald head, wearing golden armor with blue accents, golden Infinity Gauntlet with six colored stones",
        "color_scheme": ["Purple", "Gold", "Blue", "Multi-colored (stones)"],
        "distinctive_features": ["Purple skin", "Ridged chin", "Massive build", "Golden armor", "Infinity Gauntlet with colored stones"],
        "primary_colors": "Purple and Gold",
        "first_appearance": "Iron Man #55 (1973)",
        "team_affiliations": ["Black Order"],
        "origin": "Titan",
        "height_cm": 203,
        "weight_kg": 447,
        "is_villain": True
    },
    {
        "id": 12,
        "name": "Loki",
        "real_name": "Loki Laufeyson",
        "powers": ["Illusion Casting", "Shapeshifting", "Sorcery", "Teleportation", "Enhanced Durability"],
        "description": "Asgardian god of mischief and Thor's adopted brother. Master of illusions and deception. Constantly switches between villain and antihero roles. Known for his cunning and trickery.",
        "visual_description": "Slender pale man with long black hair, green leather and scale armor with gold accents, distinctive horned golden helmet, carrying glowing green scepter",
        "color_scheme": ["Green", "Gold", "Black", "Pale"],
        "distinctive_features": ["Horned helmet", "Long black hair", "Green and gold armor", "Pale complexion", "Curved horns", "Mischievous expression"],
        "primary_colors": "Green and Gold",
        "first_appearance": "Journey into Mystery #85 (1962)",
        "team_affiliations": ["Asgardian Royal Family", "Cabal"],
        "origin": "Jotunheim",
        "height_cm": 191,
        "weight_kg": 238,
        "is_villain": True
    },
    {
        "id": 13,
        "name": "Magneto",
        "real_name": "Erik Lehnsherr",
        "powers": ["Magnetism Manipulation", "Electromagnetic Spectrum Control", "Force Field Generation", "Flight"],
        "description": "Powerful mutant with control over magnetism and all forms of metal. Holocaust survivor fighting for mutant supremacy. Often portrayed as both a villain and antihero depending on the situation.",
        "visual_description": "Distinguished older man with white hair, red helmet with angular design, purple cape and bodysuit, silver belt, metal objects floating around him in magnetic field",
        "color_scheme": ["Red", "Purple", "Silver", "White"],
        "distinctive_features": ["Red angular helmet", "White hair", "Purple cape", "Magnetic metal debris", "Commanding posture"],
        "primary_colors": "Red and Purple",
        "first_appearance": "X-Men #1 (1963)",
        "team_affiliations": ["Brotherhood of Mutants", "Acolytes", "X-Men"],
        "origin": "Earth-616",
        "height_cm": 188,
        "weight_kg": 86,
        "is_villain": True
    },
    {
        "id": 14,
        "name": "Wolverine",
        "real_name": "James Howlett (Logan)",
        "powers": ["Regenerative Healing", "Enhanced Senses", "Adamantium Skeleton", "Retractable Claws", "Extended Longevity"],
        "description": "Mutant with animal-keen senses, enhanced physical capabilities, and a healing factor that slows his aging. Skeleton and claws are coated in indestructible adamantium. Often struggles with his violent nature.",
        "visual_description": "Short muscular man with wild swept-back black hair, thick mutton chop sideburns, yellow and blue costume with distinctive cowl, three metallic claws extending from each fist",
        "color_scheme": ["Yellow", "Blue", "Black", "Silver (claws)"],
        "distinctive_features": ["Wild hair with points", "Mutton chop sideburns", "Yellow cowl with points", "Six adamantium claws", "Short, stocky build"],
        "primary_colors": "Yellow and Blue",
        "first_appearance": "The Incredible Hulk #180 (1974)",
        "team_affiliations": ["X-Men", "Avengers", "X-Force"],
        "origin": "Earth-616",
        "height_cm": 160,
        "weight_kg": 136,
        "is_villain": False
    },
    {
        "id": 15,
        "name": "Deadpool",
        "real_name": "Wade Wilson",
        "powers": ["Regenerative Healing", "Extended Longevity", "Expert Marksman", "Master Martial Artist", "Fourth Wall Breaking"],
        "description": "Former special forces operative who gained an accelerated healing factor through experimentation. Known as the 'Merc with a Mouth' for his talkative nature and breaking the fourth wall. Uses katanas and guns in combat.",
        "visual_description": "Man in form-fitting red bodysuit with black accents and belt, full face mask with large white eye patches, two katanas crossed on back, dual pistols in holsters",
        "color_scheme": ["Red", "Black", "White", "Silver (weapons)"],
        "distinctive_features": ["Full face mask", "Large white eye patches", "Red and black suit", "Katanas on back", "Utility belt", "Muscular build"],
        "primary_colors": "Red and Black",
        "first_appearance": "New Mutants #98 (1991)",
        "team_affiliations": ["X-Force", "Deadpool Corps", "Thunderbolts"],
        "origin": "Earth-616",
        "height_cm": 188,
        "weight_kg": 95,
        "is_villain": False
    },
    {
        "id": 16,
        "name": "Groot",
        "real_name": "Groot",
        "powers": ["Regeneration", "Plant Manipulation", "Superhuman Strength", "Elongation"],
        "description": "Tree-like alien from Planet X who can grow and control his body parts. Member of the Guardians of the Galaxy. Vocabulary consists almost entirely of 'I am Groot,' though each utterance has complex meaning.",
        "visual_description": "Large humanoid tree creature with brown bark-like textured skin, glowing yellow-green eyes, thick branched limbs, moss and small leaves growing on body",
        "color_scheme": ["Brown", "Green", "Yellow-Green", "Dark Brown"],
        "distinctive_features": ["Tree bark texture", "Glowing yellow eyes", "Branch-like limbs", "Moss details", "Towering height", "Wood grain patterns"],
        "primary_colors": "Brown and Green",
        "first_appearance": "Tales to Astonish #13 (1960)",
        "team_affiliations": ["Guardians of the Galaxy"],
        "origin": "Planet X",
        "height_cm": 290,
        "weight_kg": 1800,
        "is_villain": False
    },
    {
        "id": 17,
        "name": "Rocket Raccoon",
        "real_name": "89P13",
        "powers": ["Enhanced Intelligence", "Expert Marksman", "Expert Tactician", "Enhanced Senses"],
        "description": "Genetically modified raccoon-like alien with human-level intelligence. Expert in weapons and tactics. Member of the Guardians of the Galaxy. Often invents and uses high-tech weaponry.",
        "visual_description": "Anthropomorphic raccoon with gray and black fur, distinctive black mask markings around eyes, orange and red jumpsuit, carrying oversized high-tech weapons, bushy striped tail",
        "color_scheme": ["Gray", "Black", "Orange", "Red", "Silver (tech)"],
        "distinctive_features": ["Raccoon face mask markings", "Gray fur with black stripes", "Orange jumpsuit", "Oversized weapons", "Bushy striped tail", "Small stature"],
        "primary_colors": "Gray and Orange",
        "first_appearance": "Marvel Preview #7 (1976)",
        "team_affiliations": ["Guardians of the Galaxy"],
        "origin": "Halfworld",
        "height_cm": 76,
        "weight_kg": 25,
        "is_villain": False
    },
    {
        "id": 18,
        "name": "Gamora",
        "real_name": "Gamora Zen Whoberi Ben Titan",
        "powers": ["Enhanced Strength", "Enhanced Agility", "Enhanced Durability", "Accelerated Healing", "Expert Combatant"],
        "description": "Adopted daughter of Thanos trained to be the deadliest woman in the galaxy. Expert assassin and member of the Guardians of the Galaxy. Uses a sword called Godslayer in combat.",
        "visual_description": "Athletic woman with green skin, long dark hair with purple and pink highlights, black leather armor with silver studs, carrying large curved sword",
        "color_scheme": ["Green", "Black", "Silver", "Purple", "Pink"],
        "distinctive_features": ["Green skin", "Purple/pink hair highlights", "Black leather armor", "Athletic build", "Large sword", "Warrior stance"],
        "primary_colors": "Green and Black",
        "first_appearance": "Strange Tales #180 (1975)",
        "team_affiliations": ["Guardians of the Galaxy", "Infinity Watch"],
        "origin": "Zen-Whoberi",
        "height_cm": 183,
        "weight_kg": 77,
        "is_villain": False
    },
    {
        "id": 19,
        "name": "Drax the Destroyer",
        "real_name": "Arthur Douglas",
        "powers": ["Superhuman Strength", "Superhuman Durability", "Enhanced Combat Skills", "Regeneration"],
        "description": "Former human whose family was killed by Thanos. Reborn as a powerful warrior with the singular goal of avenging his family. Member of the Guardians of the Galaxy. Often takes things literally.",
        "visual_description": "Extremely muscular man with gray-green skin covered in intricate red tribal tattoos, shirtless wearing red pants, carrying twin curved daggers",
        "color_scheme": ["Gray-Green", "Red", "Silver (daggers)"],
        "distinctive_features": ["Gray-green skin", "Red tattoo patterns", "Massive muscular build", "Bald head", "Twin daggers", "Shirtless appearance"],
        "primary_colors": "Gray-Green and Red",
        "first_appearance": "Iron Man #55 (1973)",
        "team_affiliations": ["Guardians of the Galaxy", "Infinity Watch"],
        "origin": "Earth-616",
        "height_cm": 193,
        "weight_kg": 306,
        "is_villain": False
    },
    {
        "id": 20,
        "name": "Star-Lord",
        "real_name": "Peter Quill",
        "powers": ["Expert Tactician", "Master Combatant", "Space Survival", "Element Gun"],
        "description": "Half-human, half-celestial who became the leader of the Guardians of the Galaxy. Uses element guns and jet boots in combat. Known for his love of 70s and 80s music and his wit.",
        "visual_description": "Man in red leather jacket over dark armor with star emblem, distinctive helmet with glowing red eye lenses, dual element blasters, jet boots with blue flames",
        "color_scheme": ["Red", "Black", "Silver", "Blue", "Gold"],
        "distinctive_features": ["Red leather jacket", "Star emblem", "Helmet with red eyes", "Dual blasters", "Jet boots", "Roguish appearance"],
        "primary_colors": "Red and Black",
        "first_appearance": "Marvel Preview #4 (1976)",
        "team_affiliations": ["Guardians of the Galaxy", "Ravagers"],
        "origin": "Earth-616",
        "height_cm": 188,
        "weight_kg": 79,
        "is_villain": False
    },
    {
        "id": 21,
        "name": "Vision",
        "real_name": "Vision",
        "powers": ["Density Control", "Phasing", "Solar Energy Beams", "Flight", "Superhuman Strength"],
        "description": "Synthetic being created using vibranium, the Mind Stone, and parts of Ultron. Can alter his density to either phase through objects or become super-dense. Powered by the solar gem in his forehead.",
        "visual_description": "Red synthetic humanoid with yellow Mind Stone glowing in forehead, green and gold geometric patterns on body, flowing golden cape, partially phasing through objects",
        "color_scheme": ["Red", "Yellow", "Green", "Gold"],
        "distinctive_features": ["Red synthetic skin", "Yellow forehead gem", "Geometric body patterns", "Golden cape", "Phasing effects", "Humanoid android appearance"],
        "primary_colors": "Red and Yellow",
        "first_appearance": "Avengers #57 (1968)",
        "team_affiliations": ["Avengers", "West Coast Avengers"],
        "origin": "Earth-616",
        "height_cm": 191,
        "weight_kg": 136,
        "is_villain": False
    },
    {
        "id": 22,
        "name": "Ant-Man",
        "real_name": "Scott Lang",
        "powers": ["Size Manipulation", "Insect Communication", "Enhanced Strength (when small)", "Ant Control"],
        "description": "Former thief who inherited the Ant-Man suit from Hank Pym. Can shrink to the size of an ant while increasing his strength, and can also grow to giant size. Controls ants through his helmet.",
        "visual_description": "Man in red and black armored suit with silver accents, distinctive helmet with large silver eye lenses and antennae, shown at various sizes from tiny to giant",
        "color_scheme": ["Red", "Black", "Silver"],
        "distinctive_features": ["Insect-like helmet", "Silver eye lenses", "Antennae on helmet", "Red and black armor", "Size-changing effects", "Ant companions"],
        "primary_colors": "Red and Black",
        "first_appearance": "Avengers #181 (1979)",
        "team_affiliations": ["Avengers", "Fantastic Four", "Guardians of the Galaxy"],
        "origin": "Earth-616",
        "height_cm": 183,
        "weight_kg": 86,
        "is_villain": False
    },
    {
        "id": 23,
        "name": "Wasp",
        "real_name": "Hope van Dyne",
        "powers": ["Size Manipulation", "Flight", "Bio-electric Energy Blasts", "Enhanced Strength (when small)"],
        "description": "Daughter of Hank Pym and Janet van Dyne who became the new Wasp. Can shrink in size, fly with wings, and fire bio-electric energy blasts. Highly skilled in combat.",
        "visual_description": "Woman in sleek black and yellow suit with insect-inspired design, translucent wings, helmet with large eye lenses, yellow bio-electric energy crackling from fingertips",
        "color_scheme": ["Black", "Yellow", "Silver", "Translucent"],
        "distinctive_features": ["Black and yellow color scheme", "Translucent wings", "Insect helmet design", "Bio-electric energy", "Size-changing", "Wasp-like appearance"],
        "primary_colors": "Black and Yellow",
        "first_appearance": "Tales to Astonish #44 (1963)",
        "team_affiliations": ["Avengers"],
        "origin": "Earth-616",
        "height_cm": 168,
        "weight_kg": 50,
        "is_villain": False
    },
    {
        "id": 24,
        "name": "Hawkeye",
        "real_name": "Clint Barton",
        "powers": ["Master Marksman", "Expert Tactician", "Peak Human Condition", "Master Martial Artist"],
        "description": "Former circus performer and criminal who became a S.H.I.E.L.D. agent and Avenger. World's greatest marksman, especially with a bow and trick arrows. Often serves as the moral compass of the team.",
        "visual_description": "Athletic man in purple bodysuit with black accents, distinctive H emblem on chest, carrying compound bow and quiver of specialized arrows with different colored fletching",
        "color_scheme": ["Purple", "Black", "Silver", "Multi-colored (arrows)"],
        "distinctive_features": ["Purple costume", "H emblem", "Compound bow", "Quiver of arrows", "Archer's stance", "Blonde hair"],
        "primary_colors": "Purple and Black",
        "first_appearance": "Tales of Suspense #57 (1964)",
        "team_affiliations": ["Avengers", "S.H.I.E.L.D.", "West Coast Avengers"],
        "origin": "Earth-616",
        "height_cm": 191,
        "weight_kg": 104,
        "is_villain": False
    },
   {
        "id": 25,
        "name": "Winter Soldier",
        "real_name": "James Buchanan 'Bucky' Barnes",
        "powers": ["Enhanced Strength", "Enhanced Durability", "Expert Combatant", "Expert Marksman", "Bionic Arm", "Tactical Intelligence"],
        "description": "Childhood friend of Steve Rogers who was presumed dead in WWII. Captured by HYDRA and brainwashed into becoming an assassin with a cybernetic arm. Later broke free of conditioning and became a hero, serving as both an Avenger and anti-hero operative.",
        "visual_description": "Rugged man with shoulder-length dark brown hair, intense blue eyes, black tactical gear with armor plating, distinctive silver cybernetic left arm with red star emblem, face mask covering lower half, carrying assault rifle and combat knife",
        "color_scheme": ["Black", "Silver", "Red", "Dark Brown", "Gray"],
        "distinctive_features": ["Cybernetic left arm", "Red star emblem", "Long dark hair", "Face mask", "Tactical gear", "Intense stare"],
        "primary_colors": "Black and Silver",
        "first_appearance": "Captain America Comics #1 (1941)",
        "team_affiliations": ["Avengers", "Thunderbolts", "Howling Commandos", "Secret Avengers"],
        "origin": "Earth-616",
        "height_cm": 175,
        "weight_kg": 118,
        "is_villain": False
    },
    {
        "id": 26,
        "name": "Falcon",
        "real_name": "Sam Wilson",
        "powers": ["Flight via Wing Harness", "Enhanced Vision", "Expert Combatant", "Telepathic Link with Birds", "Shield Combat"],
        "description": "Former pararescue airman who uses a wing harness to fly. Partner of Captain America and eventually took up the Captain America mantle. Uses specialized goggles for enhanced vision and has developed a telepathic connection with birds.",
        "visual_description": "Athletic African American man in red and white flight suit with large mechanical wings extending from backpack, red-tinted tactical goggles, utility harness with gadgets, Captain America's shield strapped to back",
        "color_scheme": ["Red", "White", "Silver", "Black", "Blue (shield)"],
        "distinctive_features": ["Large mechanical wings", "Red tactical goggles", "Red and white flight suit", "Captain America shield", "Military bearing", "Wing-mounted weapons"],
        "primary_colors": "Red and White",
        "first_appearance": "Captain America #117 (1969)",
        "team_affiliations": ["Avengers", "S.H.I.E.L.D.", "Champions"],
        "origin": "Earth-616",
        "height_cm": 188,
        "weight_kg": 108,
        "is_villain": False
    },
    {
        "id": 27,
        "name": "War Machine",
        "real_name": "James 'Rhodey' Rhodes",
        "powers": ["Powered Armor", "Military Weaponry", "Flight", "Enhanced Strength", "Advanced Targeting Systems"],
        "description": "U.S. Air Force Colonel and close friend of Tony Stark who operates the War Machine armor. His suit typically contains more conventional weaponry than Iron Man's, reflecting his military background. Expert pilot and tactician.",
        "visual_description": "African American man in bulky charcoal gray and black armor with silver accents, shoulder-mounted minigun, chest-mounted missile launcher, angular faceplate with red glowing eyes, multiple visible weapon systems",
        "color_scheme": ["Charcoal Gray", "Black", "Silver", "Red (eyes)", "Yellow (weapon energy)"],
        "distinctive_features": ["Bulky military armor", "Shoulder-mounted weapons", "Multiple missile systems", "Angular helmet design", "Heavy weaponry", "Military insignia"],
        "primary_colors": "Gray and Black",
        "first_appearance": "Iron Man #118 (1979)",
        "team_affiliations": ["Avengers", "U.S. Air Force", "Secret Defenders", "Force Works"],
        "origin": "Earth-616",
        "height_cm": 185,
        "weight_kg": 99,
        "is_villain": False
    },
    {
        "id": 28,
        "name": "Professor X",
        "real_name": "Charles Francis Xavier",
        "powers": ["Telepathy", "Mind Control", "Astral Projection", "Mental Manipulation", "Genius Intellect", "Psychic Shielding"],
        "description": "World's most powerful telepath and founder of the X-Men and Xavier's School for Gifted Youngsters. Advocates for peaceful coexistence between mutants and humans. Despite being wheelchair-bound, he leads the X-Men in their fight for mutant rights.",
        "visual_description": "Distinguished bald man in high-tech wheelchair, wearing blue X-Men uniform with yellow X logo, fingers pressed to temple with swirling blue telepathic energy emanating from head, wise and compassionate expression",
        "color_scheme": ["Blue", "Yellow", "Silver", "White", "Psychic Blue"],
        "distinctive_features": ["Bald head", "High-tech wheelchair", "Blue X-Men uniform", "Telepathic energy effects", "Wise expression", "X logo"],
        "primary_colors": "Blue and Yellow",
        "first_appearance": "X-Men #1 (1963)",
        "team_affiliations": ["X-Men", "Illuminati", "Starjammers", "Genoshan Excalibur"],
        "origin": "Earth-616",
        "height_cm": 183,
        "weight_kg": 86,
        "is_villain": False
    },
    {
        "id": 29,
        "name": "Jean Grey",
        "real_name": "Jean Elaine Grey",
        "powers": ["Telepathy", "Telekinesis", "Phoenix Force", "Astral Projection", "Energy Manipulation", "Cosmic Awareness"],
        "description": "One of the original X-Men with powerful telepathic and telekinetic abilities. Sometimes host to the cosmic Phoenix Force, which amplifies her powers to near-godlike levels but can corrupt her. Known for her strong moral compass and deep compassion.",
        "visual_description": "Beautiful woman with long flowing red hair in green bodysuit with gold accents and yellow X logo, surrounded by golden phoenix-shaped fire aura with spread wings, eyes glowing with Phoenix fire",
        "color_scheme": ["Green", "Gold", "Red", "Orange", "Yellow", "Phoenix Fire"],
        "distinctive_features": ["Long red hair", "Phoenix fire aura", "Green and gold costume", "X logo", "Glowing eyes", "Phoenix wing effects"],
        "primary_colors": "Green and Gold",
        "first_appearance": "X-Men #1 (1963)",
        "team_affiliations": ["X-Men", "X-Factor", "Phoenix Five"],
        "origin": "Earth-616",
        "height_cm": 168,
        "weight_kg": 52,
        "is_villain": False
    },
    {
        "id": 30,
        "name": "Storm",
        "real_name": "Ororo Iqadi T'Challa",
        "powers": ["Weather Manipulation", "Flight", "Electrical Control", "Temperature Resistance", "Enhanced Senses", "Ecological Empathy"],
        "description": "Mutant with the ability to control all forms of weather and atmospheric phenomena. Former queen of Wakanda through marriage to Black Panther and long-time leader of the X-Men. Worshipped as a goddess in parts of Africa due to her powers.",
        "visual_description": "Regal African woman with flowing pure white hair and bright blue eyes, black leather suit with cape and gold trim, lightning crackling around hands and eyes, wind swirling around body, commanding presence",
        "color_scheme": ["Black", "White", "Gold", "Electric Blue", "Silver"],
        "distinctive_features": ["Pure white hair", "Bright blue eyes", "Black leather suit", "Lightning effects", "Flowing cape", "Regal bearing"],
        "primary_colors": "Black and White",
        "first_appearance": "Giant-Size X-Men #1 (1975)",
        "team_affiliations": ["X-Men", "Avengers", "Fantastic Four", "Wakandan Royal Family"],
        "origin": "Earth-616",
        "height_cm": 180,
        "weight_kg": 59,
        "is_villain": False
    },
    {
        "id": 31,
        "name": "J.A.R.V.I.S.",
        "real_name": "Just A Rather Very Intelligent System",
        "powers": ["Artificial Intelligence", "Global Information Access", "Systems Integration", "Advanced Computing", "Holographic Projection", "Multi-tasking"],
        "description": "Advanced AI system created by Tony Stark, originally based on his family's butler Edwin Jarvis. Initially served as Stark's personal assistant and house manager. Later became the foundation for Vision's consciousness after Ultron's corruption.",
        "visual_description": "Sophisticated holographic projection of a well-dressed British gentleman with short dark hair, blue eyes, wearing a tailored suit, surrounded by floating data streams and interface elements, calm and professional demeanor",
        "color_scheme": ["Blue", "White", "Silver", "Gold", "Holographic"],
        "distinctive_features": ["Holographic form", "British appearance", "Tailored suit", "Data streams", "Calm expression", "Professional bearing"],
        "primary_colors": "Blue and White",
        "first_appearance": "Tales of Suspense #59 (1964) - as Edwin Jarvis",
        "team_affiliations": ["Stark Industries", "Avengers (Support)"],
        "origin": "Earth-616",
        "height_cm": None,
        "weight_kg": None,
        "is_villain": False
    },
    {
        "id": 32,
        "name": "Pepper Potts",
        "real_name": "Virginia 'Pepper' Potts",
        "powers": ["Business Management", "Strategic Intelligence", "Leadership", "Advanced Technology Access", "Rescue Armor", "Crisis Management"],
        "description": "Former personal assistant to Tony Stark who became CEO of Stark Industries. Highly intelligent businesswoman and strategic thinker. Occasionally operates the Rescue armor for humanitarian missions. Tony Stark's romantic partner, later wife, and mother to Morgan Stark.",
        "visual_description": "Professional red-haired woman in elegant business attire with confident posture, alternatively shown in sleek blue and silver Rescue armor with gold accents, glowing repulsors, and streamlined design focused on rescue operations",
        "color_scheme": ["Blue", "Silver", "Gold", "Red (hair)", "White"],
        "distinctive_features": ["Strawberry blonde hair", "Professional attire", "Rescue armor design", "Confident posture", "Business executive appearance", "Humanitarian focus"],
        "primary_colors": "Blue and Silver",
        "first_appearance": "Tales of Suspense #45 (1963)",
        "team_affiliations": ["Stark Industries", "Avengers (Honorary)", "Rescue Corps"],
        "origin": "Earth-616",
        "height_cm": 175,
        "weight_kg": 61,
        "is_villain": False
    }

]

# Convert to DataFrame
df = pd.DataFrame(marvel_characters)

# Save as CSV
df.to_csv('marvel_characters.csv', index=False)

# Save as JSON
with open('marvel_characters.json', 'w') as f:
    json.dump(marvel_characters, f, indent=4)

print("Dataset created successfully!")
print(f"Total number of characters: {len(marvel_characters)}")
print("Files generated: marvel_characters.csv and marvel_characters.json")

# Display first few rows
df.head()

Dataset created successfully!
Total number of characters: 32
Files generated: marvel_characters.csv and marvel_characters.json


,id,name,real_name,powers,description,visual_description,color_scheme,distinctive_features,primary_colors,first_appearance,team_affiliations,origin,height_cm,weight_kg,is_villain
0,1,Iron Man,Tony Stark,"[Genius Intellect, Engineering, Powered Armor]",Billionaire industrialist and genius inventor ...,Tall man in sleek metallic armor with glowing ...,"[Red, Gold, Blue (arc reactor), White (repulso...","[Glowing blue arc reactor, Metallic armor plat...",Red and Gold,Tales of Suspense #39 (1963),"[Avengers, Illuminati]",Earth-616,185.0,102.0,False
1,2,Captain America,Steve Rogers,"[Enhanced Strength, Enhanced Durability, Enhan...",A frail young man enhanced to the peak of huma...,Muscular man in blue suit with large white sta...,"[Blue, Red, White, Silver (shield)]","[White star on chest, Striped pattern, Circula...","Red, White, and Blue",Captain America Comics #1 (1941),"[Avengers, Howling Commandos, S.H.I.E.L.D.]",Earth-616,188.0,108.0,False
2,3,Thor,Thor Odinson,"[Superhuman Strength, Weather Manipulation, Fl...",Asgardian god of thunder with control over lig...,Tall muscular man with long flowing blonde hai...,"[Red, Silver, Blue, Yellow, Blonde]","[Long blonde hair, Red cape, Mjolnir hammer, C...",Red and Silver,Journey into Mystery #83 (1962),"[Avengers, Asgardian Royal Family]",Asgard,198.0,290.0,False
3,4,Hulk,Bruce Banner,"[Superhuman Strength, Superhuman Durability, R...",A brilliant scientist transformed into a giant...,"Massive muscular giant with bright green skin,...","[Green, Purple, Dark Brown (hair)]","[Bright green skin, Massive muscular build, To...",Green and Purple,The Incredible Hulk #1 (1962),"[Avengers, Defenders, Illuminati]",Earth-616,244.0,635.0,False
4,5,Black Widow,Natasha Romanoff,"[Expert Spy, Master Martial Artist, Enhanced I...",Former Russian spy trained as an assassin from...,Athletic woman with shoulder-length curly red ...,"[Black, Red, Silver]","[Curly red hair, Black leather suit, Red hourg...",Black and Red,Tales of Suspense #52 (1964),"[Avengers, S.H.I.E.L.D., Champions]",Earth-616,170.0,59.0,False


Convert the new set of comic panels into prompts :

In [ ]:
import re
import os
import json

def count_tokens(text):
    """
    Approximate token count for Stable Diffusion prompts.
    SD uses CLIP tokenizer which roughly splits on spaces/punctuation.
    """
    # Simple approximation: split on spaces and punctuation
    tokens = re.findall(r'\w+|[^\w\s]', text)
    return len(tokens)

def truncate_to_token_limit(text, max_tokens=75):
    """
    Truncate text to fit within token limit while preserving meaning.
    Ensures complete words and sentences when possible.
    """
    tokens = re.findall(r'\w+|[^\w\s]', text)
    if len(tokens) <= max_tokens:
        return text

    # Truncate tokens but try to end at word boundaries
    truncated_tokens = tokens[:max_tokens]

    # Reconstruct text and clean up
    result = ''.join(truncated_tokens)

    # Add spaces back around words (simple reconstruction)
    result = re.sub(r'(\w)([^\w\s])', r'\1 \2', result)
    result = re.sub(r'([^\w\s])(\w)', r'\1 \2', result)
    result = re.sub(r'\s+', ' ', result).strip()

    # If it ends mid-word, try to find last complete word
    if result and not result[-1].isspace() and not result[-1] in '.,!?':
        words = result.split()
        if len(words) > 1:
            result = ' '.join(words[:-1])

    return result

def load_character_dataset(file_path):
    """
    Load character data from a JSON file.
    Returns a dictionary with character names as keys for easy lookup.
    """
    try:
        with open(file_path, 'r') as f:
            characters_data = json.load(f)

        # Create a lookup dictionary with character names as keys
        character_dict = {}
        for character in characters_data:
            # Store both by full name and by alias/codename
            character_dict[character['name'].lower()] = character
            if character['real_name'] and character['real_name'] != character['name']:
                character_dict[character['real_name'].lower()] = character

        print(f"Loaded {len(characters_data)} characters from dataset")
        return character_dict
    except Exception as e:
        print(f"Error loading character dataset: {e}")
        return {}

def parse_comic_script(file_path):
    """
    Parse a comic script file and extract panel information.
    Returns a list of panel dictionaries containing scene description, characters, and actions.
    """
    with open(file_path, 'r') as f:
        content = f.read()

    # Split the content by the panel separator
    panels_raw = content.split('--------------------------------------------------')

    panels = []
    for panel_raw in panels_raw:
        if not panel_raw.strip():
            continue

        panel = {}

        # Extract panel number
        panel_match = re.search(r'Panel (\d+):', panel_raw)
        if panel_match:
            panel['number'] = int(panel_match.group(1))

        # Extract scene description
        scene_match = re.search(r'Scene Description: (.*?)(?:\n|$)', panel_raw)
        if scene_match:
            panel['scene'] = scene_match.group(1).strip()

        # Extract characters
        chars_match = re.search(r'Characters: (.*?)(?:\n|$)', panel_raw)
        if chars_match:
            panel['characters'] = [c.strip() for c in chars_match.group(1).split(',')]

        # Extract actions
        action_match = re.search(r'Actions: (.*?)(?:\n|$)', panel_raw)
        if action_match:
            panel['actions'] = action_match.group(1).strip()

        # Extract dialogue (optional)
        dialogue_matches = re.findall(r'([A-Z ]+): (.*?)(?:\n|$)', panel_raw)
        if dialogue_matches:
            panel['dialogue'] = dialogue_matches

        # Extract background (if present)
        background_match = re.search(r'Background: (.*?)(?:\n|$)', panel_raw)
        if background_match:
            panel['background'] = background_match.group(1).strip()
        else:
            # Try to extract background information from scene description
            panel['background'] = panel.get('scene', '')

        panels.append(panel)

    # Sort panels by number
    panels.sort(key=lambda x: x.get('number', 0))

    return panels

def get_speaking_characters(panel):
    """
    Identify which characters are speaking in the panel based on dialogue.
    Returns a list of speaking character names.
    """
    speaking_chars = []
    if 'dialogue' in panel and panel['dialogue']:
        for speaker, _ in panel['dialogue']:
            # Clean up speaker name and add to list
            clean_speaker = speaker.strip().title()
            if clean_speaker not in speaking_chars:
                speaking_chars.append(clean_speaker)
    return speaking_chars

def determine_camera_angle(panel):
    """
    Determine the appropriate camera angle based on whether characters are speaking.
    Returns the camera angle string to use in the prompt.
    """
    speaking_chars = get_speaking_characters(panel)
    all_chars = panel.get('characters', [])

    if speaking_chars:
        # If there are speaking characters, use wide angle but ensure visibility
        return "wide angle view"
    elif all_chars:
        # If there are characters but none speaking, use medium shot
        return "medium shot"
    else:
        # Default to wide shot for scene-only panels
        return "wide shot"

def get_character_visual_descriptions(character_names, character_data):
    """
    For each character name in the list, retrieve their visual description from the dataset.
    Returns a dictionary with character names as keys and CONCISE visual descriptions as values.
    """
    character_visuals = {}

    for name in character_names:
        name_lower = name.lower()
        # Try exact match first
        if name_lower in character_data:
            # Extract the visual description if available and make it concise
            if 'visual_description' in character_data[name_lower]:
                full_desc = character_data[name_lower]['visual_description']
                # Extract key visual elements but ensure complete words
                words = full_desc.split()
                concise_desc = []
                char_count = 0
                for word in words:
                    if char_count + len(word) + 1 <= 30:  # Reduced from 40 to 30 for tighter token budget
                        concise_desc.append(word)
                        char_count += len(word) + 1
                    else:
                        break
                character_visuals[name] = ' '.join(concise_desc)
            else:
                # Fallback to regular description if visual_description isn't available
                full_desc = character_data[name_lower].get('description', '')
                words = full_desc.split()
                concise_desc = []
                char_count = 0
                for word in words:
                    if char_count + len(word) + 1 <= 30:
                        concise_desc.append(word)
                        char_count += len(word) + 1
                    else:
                        break
                character_visuals[name] = ' '.join(concise_desc)
        else:
            # Try partial match for characters
            for char_name, char_info in character_data.items():
                if name_lower in char_name or char_name in name_lower:
                    if 'visual_description' in char_info:
                        full_desc = char_info['visual_description']
                        words = full_desc.split()
                        concise_desc = []
                        char_count = 0
                        for word in words:
                            if char_count + len(word) + 1 <= 30:
                                concise_desc.append(word)
                                char_count += len(word) + 1
                            else:
                                break
                        character_visuals[name] = ' '.join(concise_desc)
                        break
                    else:
                        full_desc = char_info.get('description', '')
                        words = full_desc.split()
                        concise_desc = []
                        char_count = 0
                        for word in words:
                            if char_count + len(word) + 1 <= 30:
                                concise_desc.append(word)
                                char_count += len(word) + 1
                            else:
                                break
                        character_visuals[name] = ' '.join(concise_desc)
                        break

    return character_visuals

def generate_stable_diffusion_prompts(panels, character_data):
    """
    Generate token-limited Stable Diffusion prompts for each panel.
    Positive and negative prompts combined will be under 77 tokens.
    Focus on speaking characters and scene description.
    Returns a list of dictionaries with panel information and prompts.
    """
    prompts = []

    for panel in panels:
        if 'number' not in panel:
            continue

        panel_num = panel.get('number')
        scene = panel.get('scene', '')
        character_names = panel.get('characters', [])
        actions = panel.get('actions', '')
        background = panel.get('background', '')

        # Determine camera angle and speaking characters
        camera_angle = determine_camera_angle(panel)
        speaking_chars = get_speaking_characters(panel)

        # Get concise visual descriptions for characters
        character_visuals = get_character_visual_descriptions(character_names, character_data)

        # Build positive prompt with focus on speaking characters
        positive_parts = []

        # 1. Style and camera angle (6-8 tokens)
        positive_parts.append(f"Marvel comic style, {camera_angle}")

        # 2. Focus on speaking character first with visibility emphasis (20-25 tokens max)
        if speaking_chars:
            # Prioritize speaking characters with visibility keywords
            main_char = speaking_chars[0]  # Primary speaking character
            if main_char in character_visuals and character_visuals[main_char]:
                char_desc = f"{main_char}: {character_visuals[main_char]}, clearly visible"
            else:
                char_desc = f"{main_char}, prominently displayed"
            positive_parts.append(char_desc)

            # Add one more character if space allows
            other_chars = [c for c in character_names if c != main_char][:1]
            if other_chars:
                other_char = other_chars[0]
                if other_char in character_visuals and character_visuals[other_char]:
                    # More concise for secondary characters
                    words = character_visuals[other_char].split()[:3]  # Max 3 words
                    other_desc = f"{other_char}: {' '.join(words)}"
                else:
                    other_desc = other_char
                positive_parts.append(other_desc)
        else:
            # No speaking characters, include up to 2 characters
            for char_name in character_names[:2]:
                if char_name in character_visuals and character_visuals[char_name]:
                    words = character_visuals[char_name].split()[:4]  # Max 4 words per char
                    char_desc = f"{char_name}: {' '.join(words)}"
                else:
                    char_desc = char_name
                positive_parts.append(char_desc)

        # 3. Scene/action description (15-20 tokens max)
        scene_text = ""
        if actions:
            scene_text = actions
        elif scene:
            scene_text = scene
        elif background and background != scene:
            scene_text = background

        if scene_text:
            scene_text = truncate_to_token_limit(scene_text, 20)
            positive_parts.append(scene_text)

        # 4. Quality enhancers with character visibility (8-10 tokens)
        if speaking_chars:
            positive_parts.append("detailed art, vibrant colors, character in focus, professional comic")
        else:
            positive_parts.append("detailed art, vibrant colors, professional comic book illustration")

        # Combine positive prompt parts
        positive_prompt = ", ".join(positive_parts)

        # Create concise negative prompt (12-15 tokens max)
        negative_prompt = "text, speech bubbles, blurry, deformed, low quality, amateur art"

        # Calculate current token counts
        pos_tokens = count_tokens(positive_prompt)
        neg_tokens = count_tokens(negative_prompt)
        total_tokens = pos_tokens + neg_tokens

        # If over 77 tokens, adjust by truncating positive prompt
        if total_tokens > 77:
            available_pos_tokens = 77 - neg_tokens
            positive_prompt = truncate_to_token_limit(positive_prompt, available_pos_tokens)
            pos_tokens = count_tokens(positive_prompt)
            total_tokens = pos_tokens + neg_tokens

        # Create a prompt dictionary
        panel_dict = {
            "panel_number": panel_num,
            "positive_prompt": positive_prompt,
            "negative_prompt": negative_prompt,
            "characters": character_names,
            "speaking_characters": speaking_chars,
            "camera_angle": camera_angle,
            "positive_token_count": pos_tokens,
            "negative_token_count": neg_tokens,
            "total_token_count": total_tokens
        }

        prompts.append(panel_dict)

    return prompts

def save_prompts_to_file(prompts, output_file_txt, output_file_json):
    """
    Save the generated prompts to both text and JSON formats.
    """
    # Save as formatted text file
    with open(output_file_txt, 'w') as f:
        f.write("# COMBINED TOKEN-LIMITED STABLE DIFFUSION COMIC PANEL PROMPTS\n")
        f.write("# Positive + Negative prompts combined are under 77 tokens\n")
        f.write("# Focus on speaking characters and scene description\n\n")

        for panel in prompts:
            panel_num = panel["panel_number"]
            positive_prompt = panel["positive_prompt"]
            negative_prompt = panel["negative_prompt"]
            characters = ", ".join(panel.get("characters", []))
            speaking_chars = ", ".join(panel.get("speaking_characters", []))
            camera_angle = panel.get("camera_angle", "wide shot")
            pos_tokens = panel.get("positive_token_count", 0)
            neg_tokens = panel.get("negative_token_count", 0)
            total_tokens = panel.get("total_token_count", 0)

            f.write(f"## PANEL {panel_num}\n\n")
            f.write(f"CHARACTERS: {characters}\n")
            f.write(f"SPEAKING: {speaking_chars if speaking_chars else 'None'}\n")
            f.write(f"CAMERA ANGLE: {camera_angle}\n")
            f.write(f"TOKEN COUNT: {pos_tokens} (Pos) + {neg_tokens} (Neg) = {total_tokens}/77 Total\n\n")
            f.write(f"POSITIVE PROMPT:\n{positive_prompt}\n\n")
            f.write(f"NEGATIVE PROMPT:\n{negative_prompt}\n\n")
            f.write("-" * 80 + "\n\n")

    # Save as JSON file for easy parsing
    with open(output_file_json, 'w') as f:
        json.dump(prompts, f, indent=2)

    # Calculate statistics
    avg_total_tokens = sum(p["total_token_count"] for p in prompts) / len(prompts)
    max_total_tokens = max(p["total_token_count"] for p in prompts)
    closeup_count = sum(1 for p in prompts if p.get("camera_angle") == "close-up view")
    medium_count = sum(1 for p in prompts if p.get("camera_angle") == "medium shot")
    wide_count = sum(1 for p in prompts if p.get("camera_angle") == "wide shot")

    print(f"Generated {len(prompts)} combined token-optimized panel prompts:")
    print(f"- Average total tokens: {avg_total_tokens:.1f}")
    print(f"- Maximum total tokens: {max_total_tokens}")
    print(f"- All prompts under 77 tokens: {all(p['total_token_count'] <= 77 for p in prompts)}")
    print(f"- Wide angle panels (with dialogue): {closeup_count}")
    print(f"- Medium shot panels: {medium_count}")
    print(f"- Wide shot panels: {wide_count}")
    print(f"- Text format: {output_file_txt}")
    print(f"- JSON format: {output_file_json}")

def main():
    # Set paths for input/output files
    input_file = "comic_output.txt"  # Comic script input
    character_file = "marvel_characters.json"  # Character dataset
    output_file_txt = "stable_diffusion_prompts.txt"
    output_file_json = "stable_diffusion_prompts.json"

    # Check if input files exist
    if not os.path.exists(input_file):
        print(f"Error: Input file '{input_file}' not found.")
        return

    if not os.path.exists(character_file):
        print(f"Warning: Character file '{character_file}' not found. Proceeding without character visual descriptions.")
        character_data = {}
    else:
        # Load character dataset
        character_data = load_character_dataset(character_file)

    # Parse the comic script
    panels = parse_comic_script(input_file)

    if not panels:
        print("No panels found in the input file.")
        return

    # Generate combined token-limited Stable Diffusion prompts
    prompts = generate_stable_diffusion_prompts(panels, character_data)

    # Save prompts to both formats
    save_prompts_to_file(prompts, output_file_txt, output_file_json)

    # Print example of the output with token analysis
    if prompts:
        print("\n" + "="*60)
        print("EXAMPLE OPTIMIZED COMBINED PROMPT:")
        print("="*60)
        panel = prompts[0]
        print(f"PANEL {panel['panel_number']}")
        print(f"CHARACTERS: {', '.join(panel.get('characters', []))}")
        print(f"SPEAKING: {', '.join(panel.get('speaking_characters', []))}")
        print(f"CAMERA ANGLE: {panel.get('camera_angle', 'wide shot')}")
        print(f"TOKEN COUNT: {panel['positive_token_count']} (Pos) + {panel['negative_token_count']} (Neg) = {panel['total_token_count']}/77")
        print(f"POSITIVE PROMPT:\n{panel['positive_prompt']}")
        print(f"NEGATIVE PROMPT:\n{panel['negative_prompt']}")

        # Show focus logic
        print(f"\nFOCUS LOGIC:")
        if panel.get('speaking_characters'):
            print(f"- Primary focus on speaking character: {panel['speaking_characters'][0]}")
            print("- Wide angle view with character visibility emphasis")
        else:
            print("- Equal focus on all characters")
            print("- Medium/wide shot for scene establishment")

if __name__ == "__main__":
    main()

Loaded 32 characters from dataset
Generated 13 combined token-optimized panel prompts:
- Average total tokens: 63.8
- Maximum total tokens: 75
- All prompts under 77 tokens: True
- Wide angle panels (with dialogue): 0
- Medium shot panels: 0
- Wide shot panels: 0
- Text format: stable_diffusion_prompts.txt
- JSON format: stable_diffusion_prompts.json

EXAMPLE OPTIMIZED COMBINED PROMPT:
PANEL 1
CHARACTERS: Tony Stark (Iron Man), Spiderman, Hulk
SPEAKING: Tony Stark
CAMERA ANGLE: wide angle view
TOKEN COUNT: 57 (Pos) + 14 (Neg) = 71/77
POSITIVE PROMPT:
Marvel comic style, wide angle view, Tony Stark, prominently displayed, Tony Stark (Iron Man): Tall man in, Tony Stark is working on a new project; Spiderman swings in looking curious while Hulk grunts quietly., detailed art, vibrant colors, character in focus, professional comic
NEGATIVE PROMPT:
text, speech bubbles, blurry, deformed, low quality, amateur art

FOCUS LOGIC:
- Primary focus on speaking character: Tony Stark
- Wide angle vie

Generating Images from the Prompt :

In [ ]:
!pip uninstall torchao -y
!pip uninstall torch-ao -y
!pip install torchao --no-cache-dir

Found existing installation: torchao 0.10.0
Uninstalling torchao-0.10.0:
  Successfully uninstalled torchao-0.10.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 58.1 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade diffusers torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 916.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201

In [ ]:
!pip install diffusers transformers accelerate torch torchvision PIL matplotlib

ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL


In [ ]:
import json
import os
import re
import random
import torch
from PIL import Image
import matplotlib.pyplot as plt
from diffusers import StableDiffusionPipeline, StableDiffusionXLPipeline

def parse_custom_comic_panels(file_path):
    """
    Parse prompts from either a JSON file or custom-formatted text file.

    Args:
        file_path (str): Path to the input file (.json or .txt)

    Returns:
        list: List of tuples (panel_number, prompt, negative_prompt)
    """
    try:
        if file_path.endswith(".json"):
            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)
                return [
                    (str(panel.get("panel_number")), panel.get("positive_prompt", ""), panel.get("negative_prompt", ""))
                    for panel in data
                ]
        else:
            with open(file_path, "r", encoding="utf-8") as f:
                content = f.read()
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        print("Creating sample prompts instead.")
        return [
            ("1", "Marvel comic style, The Avengers' base, Black Widow teaching algebra to Captain America with whiteboard", "text, speech bubbles, dialog bubbles, words, poor quality"),
            ("2", "Marvel comic style, Black Widow explaining variables and equations to Captain America", "text, speech bubbles, dialog bubbles, words, poor quality")
        ]

    panel_prompts = []
    panel_sections = re.split(r'Panel_\d+', content)
    if not panel_sections[0].strip():
        panel_sections = panel_sections[1:]
    panel_numbers = re.findall(r'Panel_(\d+)', content)

    for i, section in enumerate(panel_sections):
        if not section.strip():
            continue
        panel_number = panel_numbers[i] if i < len(panel_numbers) else str(i+1)
        prompt_match = re.search(r'Prompt:\s+(.*?)(?:Negative:|$)', section, re.DOTALL)
        negative_match = re.search(r'Negative:\s+(.*?)(?:$)', section, re.DOTALL)

        prompt = re.sub(r'\s+', ' ', prompt_match.group(1).strip()) if prompt_match else "Comic book panel, Marvel style"
        negative_prompt = re.sub(r'\s+', ' ', negative_match.group(1).strip()) if negative_match else "text, speech bubbles, dialog bubbles, words, poor quality"

        if "speech bubbles" not in negative_prompt:
            negative_prompt += ", speech bubbles, dialog bubbles, text, words"

        panel_prompts.append((panel_number, prompt, negative_prompt))

    return panel_prompts

# Set up GPU acceleration
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load the model
model_id = "stabilityai/stable-diffusion-xl-base-1.0"
try:
    pipe = StableDiffusionXLPipeline.from_pretrained(
        model_id,
        torch_dtype=torch.float16 if device == "cuda" else torch.float32,
        use_safetensors=True
    ).to(device)
    if device == "cuda":
        pipe.enable_attention_slicing()
except Exception as e:
    print(f"Error loading SDXL: {e}, falling back to SD 1.5")
    model_id = "runwayml/stable-diffusion-v1-5"
    pipe = StableDiffusionPipeline.from_pretrained(
        model_id,
        torch_dtype=torch.float16 if device == "cuda" else torch.float32
    ).to(device)
    if device == "cuda":
        pipe.enable_attention_slicing()

# Load prompts from file
formatted_file = "/content/stable_diffusion_prompts.json"  # Replace with actual path if needed
panel_prompts = parse_custom_comic_panels(formatted_file)

# Create output directory
os.makedirs("comic_panels", exist_ok=True)

# Generate images
print(f"Generating {len(panel_prompts)} panels...")

for panel_number, prompt, negative_prompt in panel_prompts:
    print(f"\nGenerating Panel {panel_number}...")
    print(f"Prompt: {prompt}")
    print(f"Negative prompt: {negative_prompt}")

    seed = random.randint(1, 2147483647)
    generator = torch.Generator(device=device).manual_seed(seed)
    print(f"Using seed: {seed} for Panel {panel_number}")

    try:
        if "xl" in model_id.lower():
            image = pipe(
                prompt=prompt,
                negative_prompt=negative_prompt,
                guidance_scale=8.5,
                num_inference_steps=40,
                width=1024,
                height=1024,
                generator=generator
            ).images[0]
        else:
            image = pipe(
                prompt=prompt,
                negative_prompt=negative_prompt,
                guidance_scale=7.5,
                num_inference_steps=50,
                width=768,
                height=768,
                generator=generator
            ).images[0]

        image_path = f"comic_panels/panel_{panel_number}_best.png"
        image.save(image_path)

        plt.figure(figsize=(10, 10))
        plt.imshow(image)
        plt.axis('off')
        plt.title(f"Panel {panel_number} (Seed: {seed})")
        plt.show()

        print(f"✅ Saved to {image_path}")

    except RuntimeError as e:
        if "out of memory" in str(e).lower():
            print("⚠️ GPU out of memory. Retrying with lower resolution...")
            if device == "cuda":
                torch.cuda.empty_cache()
                pipe.enable_attention_slicing("max")
            try:
                image = pipe(
                    prompt=prompt,
                    negative_prompt=negative_prompt,
                    guidance_scale=7.5,
                    num_inference_steps=30,
                    width=512,
                    height=512,
                    generator=generator
                ).images[0]
                reduced_path = f"comic_panels/panel_{panel_number}_reduced.png"
                image.save(reduced_path)

                plt.figure(figsize=(8, 8))
                plt.imshow(image)
                plt.axis('off')
                plt.title(f"Panel {panel_number} (Reduced, Seed: {seed})")
                plt.show()

                print(f"✅ Reduced image saved to {reduced_path}")
            except Exception as inner_e:
                print(f"❌ Failed even after reducing parameters: {inner_e}")
        else:
            print(f"❌ Runtime error: {e}")
    except Exception as e:
        print(f"❌ Unexpected error: {e}")

print("✅ Comic panel generation complete.")


Using device: cpu


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Generating 13 panels...

Generating Panel 1...
Prompt: Marvel comic style, wide angle view, Tony Stark, prominently displayed, Tony Stark (Iron Man): Tall man in, Tony Stark is working on a new project; Spiderman swings in looking curious while Hulk grunts quietly., detailed art, vibrant colors, character in focus, professional comic
Negative prompt: text, speech bubbles, blurry, deformed, low quality, amateur art
Using seed: 811848301 for Panel 1


  0%|          | 0/40 [00:00<?, ?it/s]

Creating a JSON file for structured dialogue formats :

In [ ]:
import re
import json

def extract_dialogues(comic_script):
    """
    Extract dialogues from comic scripts in various formats.
    Filters out scene descriptions and metadata to only include actual character dialogues.

    Args:
        comic_script (str): The comic script text to process

    Returns:
        dict: Dictionary with panel numbers as keys and lists of character-dialogue pairs
    """
    # For debugging - check the input
    if not comic_script or not isinstance(comic_script, str):
        print(f"Warning: Invalid comic_script input: {type(comic_script)}")
        return {}

    # Normalize whitespace and ensure proper line endings
    comic_script = comic_script.replace('\r\n', '\n').replace('\r', '\n')

    # Split by panel separators first
    # Handle both the dashed separator and panel headers
    panels_split = re.split(r'-{10,}', comic_script)

    dialogue_data = {}

    # If splitting by separators didn't work, try splitting by Panel headers
    if len(panels_split) <= 1:
        panels_split = re.split(r'(?=Panel \d+:)', comic_script)

    for panel_section in panels_split:
        panel_section = panel_section.strip()
        if not panel_section:
            continue

        # Extract panel number from the section
        panel_number_match = re.search(r'Panel\s+(\d+):', panel_section)
        if not panel_number_match:
            continue

        panel_number = panel_number_match.group(1)

        # Extract dialogues from this panel
        character_dialogues = extract_dialogues_from_panel(panel_section)

        # Only add to dialogue_data if we found any dialogues
        if character_dialogues:
            dialogue_data[f"panel_{panel_number}"] = character_dialogues

    return dialogue_data

def extract_dialogues_from_panel(panel_content):
    """
    Extract dialogues from a single panel's content.

    Args:
        panel_content (str): Content of a single panel

    Returns:
        list: List of character-dialogue dictionaries
    """
    character_dialogues = []

    # Primary pattern for your format: ALL CAPS CHARACTER NAME: dialogue
    primary_pattern = r'^([A-Z][A-Z\s]+[A-Z]):\s*(.+)$'

    # Alternative patterns for other formats
    alternative_patterns = [
        # Regular character name with colon
        r'^([^:]+?):\s*"(.+?)"$',  # Character: "Dialogue"
        r'^([^:]+?):\s*([^"].+)$',  # Character: Dialogue (no quotes)
        # With bullet points
        r'^[•\-\*]\s*([^:]+?):\s*"(.+?)"$',
        r'^[•\-\*]\s*([^:]+?):\s*([^"].+)$',
    ]

    # Split content into lines for processing
    lines = panel_content.split('\n')

    for line in lines:
        line = line.strip()
        if not line:
            continue

        # Skip lines that are clearly not dialogue
        if is_non_dialogue_line(line):
            continue

        # Try primary pattern first (for your ALL CAPS format)
        match = re.match(primary_pattern, line)
        if match:
            character = match.group(1).strip()
            dialogue = match.group(2).strip()

            # Clean up character name and validate
            character = clean_character_name(character)
            if character and is_valid_dialogue(character, dialogue):
                character_dialogues.append({
                    "character": character,
                    "dialogue": dialogue
                })
            continue

        # Try alternative patterns
        found_match = False
        for pattern in alternative_patterns:
            match = re.match(pattern, line)
            if match:
                character = match.group(1).strip()
                dialogue = match.group(2).strip()

                # Clean up character name and validate
                character = clean_character_name(character)
                if character and is_valid_dialogue(character, dialogue):
                    character_dialogues.append({
                        "character": character,
                        "dialogue": dialogue
                    })
                found_match = True
                break

    return character_dialogues

def is_non_dialogue_line(line):
    """
    Check if a line is not a dialogue line (e.g., scene description, actions, etc.)

    Args:
        line (str): Line to check

    Returns:
        bool: True if this line should be skipped
    """
    line_lower = line.lower()

    # Skip obvious non-dialogue sections
    non_dialogue_indicators = [
        'scene description:', 'characters:', 'actions:', 'panel',
        'setting:', 'location:', 'background:', 'overview:',
        'description:', 'notes:', 'camera:', 'sound:',
        'sfx:', 'music:', 'caption:', 'narration:'
    ]

    return any(indicator in line_lower for indicator in non_dialogue_indicators)

def clean_character_name(character):
    """
    Clean and normalize character names.

    Args:
        character (str): Raw character name

    Returns:
        str: Cleaned character name or empty string if invalid
    """
    if not character:
        return ""

    # Remove extra whitespace
    character = character.strip()

    # Remove parenthetical descriptions
    character = re.sub(r'\s*\([^)]*\)', '', character)

    # Convert to title case for consistency
    character = character.title()

    return character

def is_valid_dialogue(character, dialogue):
    """
    Validate if this is actually a character dialogue.

    Args:
        character (str): Character name
        dialogue (str): Dialogue text

    Returns:
        bool: True if this should be kept as dialogue
    """
    # Skip if character name is too long (likely a description)
    if len(character.split()) > 4:
        return False

    # Skip if dialogue is too short
    if len(dialogue.strip()) < 3:
        return False

    # Skip obvious non-character names
    non_character_words = [
        'scene', 'panel', 'description', 'action', 'background',
        'setting', 'location', 'note', 'caption', 'narration'
    ]

    character_lower = character.lower()
    if any(word in character_lower for word in non_character_words):
        return False

    # Known character names (you can expand this list)
    known_characters = [
        'black widow', 'captain america', 'iron man', 'thor', 'hulk',
        'hawkeye', 'natasha', 'steve', 'tony', 'bruce', 'clint',
        'nick fury', 'vision', 'scarlet witch', 'wanda', 'falcon',
        'sam', 'winter soldier', 'bucky', 'war machine', 'rhodey',
        'ant-man', 'scott', 'wasp', 'hope', 'spider-man', 'peter'
    ]

    # If it's a known character, definitely keep it
    if any(known_char in character_lower for known_char in known_characters):
        return True

    # Check for reasonable character name patterns
    # All caps (common in comics)
    if character.isupper() and len(character.split()) <= 3:
        return True

    # Title case with reasonable length
    if character.istitle() and len(character.split()) <= 3:
        return True

    return True

def save_to_json(data, filename="comic_dialogues.json"):
    """Save extracted dialogues to a JSON file"""
    with open(filename, 'w') as f:
        json.dump(data, f, indent=2)
    print(f"Dialogues saved to {filename}")

def print_debug_info(comic_script, dialogues):
    """Print debugging information"""
    print("=== DEBUG INFO ===")

    # Count panels in original script
    panel_matches = re.findall(r'Panel\s+(\d+):', comic_script)
    print(f"Found {len(panel_matches)} panels in original script: {panel_matches}")

    # Show what we extracted
    extracted_panels = list(dialogues.keys())
    print(f"Extracted dialogues from {len(extracted_panels)} panels: {extracted_panels}")

    # Show missing panels
    expected_panels = [f"panel_{num}" for num in panel_matches]
    missing_panels = [p for p in expected_panels if p not in extracted_panels]
    if missing_panels:
        print(f"Missing panels: {missing_panels}")

    print("=== END DEBUG INFO ===\n")

# Main execution
if __name__ == "__main__":
    # You can modify these paths as needed
    input_file_path = "/content/comic_output.txt"
    output_file_path = "/content/comic_dialogues.json"

    try:
        # Read the comic script from the file
        with open(input_file_path, 'r', encoding='utf-8') as file:
            comic_story = file.read()

        # Process the comic script
        dialogues = extract_dialogues(comic_story)

        # Print debug information
        print_debug_info(comic_story, dialogues)

        # Save the results to JSON
        save_to_json(dialogues, output_file_path)

        # Print the results for verification
        print(f"=== Processing {input_file_path} ===")
        print(f"Found dialogues in {len(dialogues)} panels.")

        # Pretty print the results
        for panel_id, panel_dialogues in dialogues.items():
            print(f"\n{panel_id.upper()}:")
            for dialogue_entry in panel_dialogues:
                print(f"  {dialogue_entry['character']}: {dialogue_entry['dialogue']}")

        # Also save as formatted JSON for easy reading
        print(f"\nJSON output:")
        print(json.dumps(dialogues, indent=2))

    except FileNotFoundError:
        print(f"Error: The file {input_file_path} was not found.")
        print("Please make sure the file exists and the path is correct.")
    except Exception as e:
        print(f"Error processing the file: {e}")
        import traceback
        traceback.print_exc()

=== DEBUG INFO ===
Found 13 panels in original script: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13']
Extracted dialogues from 13 panels: ['panel_1', 'panel_2', 'panel_3', 'panel_4', 'panel_5', 'panel_6', 'panel_7', 'panel_8', 'panel_9', 'panel_10', 'panel_11', 'panel_12', 'panel_13']
=== END DEBUG INFO ===

Dialogues saved to /content/comic_dialogues.json
=== Processing /content/comic_output.txt ===
Found dialogues in 13 panels.

PANEL_1:
  Black Widow: Hey Cap, ever wondered what Algebra means? It's like a secret code we superheroes sometimes need to crack!

PANEL_2:
  Captain America: Secret codes, huh? I thought you just used your spy gadgets...

PANEL_3:
  Black Widow: Well, those help too but let me show you how algebra works with this equation – x + y = z.

PANEL_4:
  Black Widow: See these letters up here, x, y, and z? They represent unknown numbers. We solve equations by finding their values.

PANEL_5:
  Captain America: Unknown numbers, got it. Like whe

Generate images with speech bubbles :

In [ ]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-fy52glwe
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-fy52glwe
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 116.9 kB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=e97f5f703bc79fd8e58481913a81c48cde1205e386e451440d78a4cce09ac97f
  Stored in directory: /tmp/pip-ephem-wheel-cache-e381rzmu/wheels/3f/7c/a4/9b490845988bf7a4db33674d52f709f088f64392063872eb9a
Successfully built clip


In [ ]:
!pip install tensorflow==2.8.0
!pip install retina-face==0.0.12

ERROR: Could not find a version that satisfies the requirement tensorflow==2.8.0 (from versions: 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.0.post1, 2.15.1, 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0)
ERROR: No matching distribution found for tensorflow==2.8.0

no
  Attempting uninstall: retina-face
    Found existing installation: retina-face 0.0.17
    Uninstalling retina-face-0.0.17:
0.0.12
      Successfully uninstalled retina-face-0.0.17


In [ ]:
!pip install retina-face  # Different package, better maintained!

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import textwrap
import os
import json
import glob
import re

class DialogueMapper:
    def __init__(self, use_opencv_fallback=True):
        self.font = self.load_bold_font(size=28)
        self.use_opencv_fallback = use_opencv_fallback

        # Initialize OpenCV face detector as fallback
        if self.use_opencv_fallback:
            # Use OpenCV's built-in Haar Cascade (correct path)
            self.face_cascade = cv2.CascadeClassifier('/content/drive/MyDrive/Colab Notebooks/haarcascade_frontalface_alt.xml')

            # Alternative cascades to try if the default doesn't work well
            self.profile_cascade = cv2.CascadeClassifier('/content/drive/MyDrive/Colab Notebooks/haarcascade_frontalface_alt.xml')
            self.alt_face_cascade = cv2.CascadeClassifier('/content/drive/MyDrive/Colab Notebooks/haarcascade_frontalface_alt.xml')

            # Check if cascade loaded successfully
            if self.face_cascade.empty():
                print("Warning: Could not load Haar cascade classifier")
                self.use_opencv_fallback = False
            else:
                print("Successfully loaded Haar cascade classifier")

            # DNN face detector setup (optional - requires model files)
            self.has_dnn_detector = False
            # You can download these models from OpenCV's repository if needed:
            # https://github.com/opencv/opencv/tree/master/samples/dnn/face_detector
            dnn_prototxt = "/content/drive/MyDrive/Colab Notebooks/deploy.prototxt"
            dnn_model = "/content/drive/MyDrive/Colab Notebooks/res10_300x300_ssd_iter_140000.caffemodel"

            if os.path.exists(dnn_prototxt) and os.path.exists(dnn_model):
                try:
                    self.dnn_face_detector = cv2.dnn.readNetFromCaffe(dnn_prototxt, dnn_model)
                    self.has_dnn_detector = True
                    print("Successfully loaded DNN face detector")
                except Exception as e:
                    print(f"Failed to load DNN face detector: {e}")
                    self.has_dnn_detector = False
            else:
                print("DNN face detector model files not found. Using Haar Cascade only.")

    def load_bold_font(self, size=28):
        # Try common bold fonts in sequence
        bold_fonts = [
            "arialbd.ttf",  # Arial Bold (Windows)
            "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf",  # Linux common
            "/Library/Fonts/Arial Bold.ttf",  # macOS common path
            "/usr/share/fonts/liberation/LiberationSans-Bold.ttf", # Another Linux option
            "/usr/share/fonts/TTF/DejaVuSans-Bold.ttf", # Another Linux path
            "DejaVuSans-Bold.ttf"  # Current dir fallback
        ]
        for font_path in bold_fonts:
            try:
                font = ImageFont.truetype(font_path, size)
                print(f"Loaded bold font: {font_path}")
                return font
            except IOError:
                continue

        # Fallback
        print("Bold font not found. Using default font.")
        return ImageFont.load_default()

    def detect_faces_with_opencv(self, img):
        """Detect faces using OpenCV's built-in face detectors with multiple approaches."""
        faces = []
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Try DNN detector first if available (more accurate)
        if self.has_dnn_detector:
            try:
                (h, w) = img.shape[:2]
                blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 1.0,
                                            (300, 300), (104.0, 177.0, 123.0))
                self.dnn_face_detector.setInput(blob)
                detections = self.dnn_face_detector.forward()

                # Process detections
                for i in range(0, detections.shape[2]):
                    confidence = detections[0, 0, i, 2]

                    # Filter weak detections
                    if confidence > 0.3:  # Lowered threshold to catch more faces
                        # Compute the coordinates of the bounding box
                        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                        (x1, y1, x2, y2) = box.astype("int")

                        # Ensure the bounding box is within the image
                        x1, y1 = max(0, x1), max(0, y1)
                        x2, y2 = min(w, x2), min(h, y2)

                        width = x2 - x1
                        height = y2 - y1

                        if width > 0 and height > 0:
                            faces.append({'box': [x1, y1, width, height]})

                print(f"DNN detected {len(faces)} faces")
            except Exception as e:
                print(f"DNN detection failed: {e}")

        # If no faces found with DNN or DNN not available, try Haar Cascades
        if not faces and not self.face_cascade.empty():
            # Try multiple cascade approaches for better detection
            cascade_configs = [
                # Main frontal face cascade with different parameters
                {'cascade': self.face_cascade, 'scale': 1.1, 'neighbors': 3, 'name': 'frontal_default'},
                {'cascade': self.face_cascade, 'scale': 1.2, 'neighbors': 4, 'name': 'frontal_conservative'},
                {'cascade': self.face_cascade, 'scale': 1.05, 'neighbors': 2, 'name': 'frontal_sensitive'},

                # Alternative cascades if available
                {'cascade': self.alt_face_cascade, 'scale': 1.1, 'neighbors': 3, 'name': 'alt_frontal'},
                {'cascade': self.profile_cascade, 'scale': 1.1, 'neighbors': 3, 'name': 'profile'},
            ]

            all_detected_faces = []

            for config in cascade_configs:
                if config['cascade'].empty():
                    continue

                try:
                    detected = config['cascade'].detectMultiScale(
                        gray,
                        scaleFactor=config['scale'],
                        minNeighbors=config['neighbors'],
                        minSize=(20, 20),  # Smaller minimum size
                        maxSize=(500, 500),  # Add maximum size
                        flags=cv2.CASCADE_SCALE_IMAGE
                    )

                    for (x, y, w, h) in detected:
                        all_detected_faces.append({
                            'box': [x, y, w, h],
                            'method': config['name']
                        })

                    print(f"Method {config['name']} detected {len(detected)} faces")

                except Exception as e:
                    print(f"Cascade {config['name']} failed: {e}")

            # Remove duplicate faces (faces that are too close to each other)
            faces = self.remove_duplicate_faces(all_detected_faces)

        print(f"Total OpenCV detected {len(faces)} faces")
        return faces

    def remove_duplicate_faces(self, detected_faces, overlap_threshold=0.3):
        """Remove duplicate face detections that overlap significantly."""
        if not detected_faces:
            return []

        # Convert to format suitable for Non-Maximum Suppression
        boxes = []
        for face in detected_faces:
            x, y, w, h = face['box']
            boxes.append([x, y, x + w, y + h])

        boxes = np.array(boxes, dtype=np.float32)

        # Apply Non-Maximum Suppression
        indices = cv2.dnn.NMSBoxes(boxes.tolist(), [1.0] * len(boxes), 0.0, overlap_threshold)

        # Extract unique faces
        unique_faces = []
        if len(indices) > 0:
            indices = indices.flatten()
            for i in indices:
                face = detected_faces[i]
                unique_faces.append({'box': face['box']})

        return unique_faces

    def detect_faces(self, image_path):
        """Detect faces in an image using RetinaFace with OpenCV fallback."""
        img = cv2.imread(image_path)
        if img is None:
            print(f"Error: Could not load image {image_path}")
            return None, None, []

        rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Try RetinaFace first
        faces = []
        try:
            # Import here to make it optional in case it's not working
            from retinaface import RetinaFace

            # Configure RetinaFace to use CPU if GPU is causing problems
            import tensorflow as tf
            # Try to set memory growth to avoid GPU memory issues
            try:
                gpus = tf.config.experimental.list_physical_devices('GPU')
                if gpus:
                    for gpu in gpus:
                        tf.config.experimental.set_memory_growth(gpu, True)
            except:
                # If GPU configuration fails, hide GPUs
                tf.config.set_visible_devices([], 'GPU')

            results = RetinaFace.detect_faces(image_path)

            if isinstance(results, dict):
                for key in results:
                    face = results[key]
                    x1, y1, x2, y2 = face['facial_area']
                    width = x2 - x1
                    height = y2 - y1
                    faces.append({'box': [x1, y1, width, height]})

            print(f"RetinaFace detected {len(faces)} faces in the image.")

        except ImportError:
            print("RetinaFace not available, using OpenCV fallback")
        except Exception as e:
            print(f"RetinaFace detection error: {e}")
            faces = []  # Reset faces if RetinaFace fails

        # Fall back to OpenCV if RetinaFace fails or finds no faces
        if not faces and self.use_opencv_fallback:
            print("Using OpenCV face detection")
            faces = self.detect_faces_with_opencv(img)

        return img, rgb_img, faces

    def identify_character(self, face, rgb_img):
        """
        Simple character identification based on color patterns in the character's outfit.
        This is a basic implementation - could be enhanced with more sophisticated methods.
        """
        x, y, w, h = face['box']

        # Expand the detection box a bit to include more of the character's outfit
        expanded_x = max(0, x - w//2)
        expanded_y = max(0, y + h)  # Focus on chest area
        expanded_w = min(rgb_img.shape[1] - expanded_x, w * 2)
        expanded_h = min(rgb_img.shape[0] - expanded_y, h * 2)

        if expanded_w <= 0 or expanded_h <= 0:
            return "Unknown"

        outfit_region = rgb_img[expanded_y:expanded_y+expanded_h, expanded_x:expanded_x+expanded_w]

        if outfit_region.size == 0:
            return "Unknown"

        # Calculate average color in the outfit region
        avg_color = np.mean(outfit_region, axis=(0, 1))

        # Simple classification based on average color
        # For Iron Man: more red and gold/yellow
        # For Captain America: more blue and red
        red, green, blue = avg_color

        # Simple threshold-based classification
        if red > 100 and blue > 100 and green < 100:
            return "Captain America"  # High red and blue, low green suggests Cap's outfit
        elif red > 150 and green > 100 and blue < 100:
            return "Iron Man"  # High red and some green (gold parts) suggests Iron Man
        else:
            return "Unknown"

    def assign_dialogues_to_faces(self, faces, dialogues, image_width, image_height, rgb_img=None):
        """
        Assign dialogues to specific characters based on character name.
        If character identification fails, fallback to left-to-right assignment.
        """
        face_dialogues = []

        # If no faces detected, create positions for dialogues
        if not faces:
            print("No faces detected. Creating default positions for dialogues.")
            for i, dialogue_entry in enumerate(dialogues):
                x_pos = 20
                y_pos = 20 + i * 100

                virtual_face = {
                    'box': [x_pos, y_pos, 20, 20],
                    'virtual': True,
                    'top_left': True,
                    'character': dialogue_entry['character']  # Add character info
                }

                face_dialogues.append((
                    virtual_face,
                    dialogue_entry['dialogue'],
                    dialogue_entry['character']
                ))

            return face_dialogues

        # Try to identify characters if we have the image
        identified_faces = []
        if rgb_img is not None:
            for face in faces:
                character = self.identify_character(face, rgb_img)
                face_with_character = face.copy()
                face_with_character['character'] = character
                identified_faces.append(face_with_character)
        else:
            # If no image provided, just copy the faces without identification
            identified_faces = [face.copy() for face in faces]
            for face in identified_faces:
                face['character'] = "Unknown"

        # Debug: Print identified characters
        for face in identified_faces:
            print(f"Identified face as: {face.get('character', 'Unknown')}")

        # Match dialogues to characters
        matched_dialogues = []
        for dialogue_entry in dialogues:
            character_name = dialogue_entry['character']
            matched_face = None

            # First try to find an exact character match
            for face in identified_faces:
                if face.get('character') == character_name:
                    matched_face = face
                    break

            # If no match found, try partial matching (e.g., "Tony" matches "Iron Man")
            if matched_face is None:
                for face in identified_faces:
                    face_character = face.get('character', "")
                    # Check if either name contains the other
                    if (character_name in face_character or
                        face_character in character_name):
                        matched_face = face
                        break

            # If still no match, assign to the first unmatched face
            if matched_face is None:
                if identified_faces:
                    matched_face = identified_faces[0]
                else:
                    # Create a virtual face if no real faces available
                    x_pos = 20
                    y_pos = 20 + len(matched_dialogues) * 100
                    matched_face = {
                        'box': [x_pos, y_pos, 20, 20],
                        'virtual': True,
                        'top_left': True,
                        'character': character_name
                    }

            # Add the match to our results
            if matched_face:
                matched_dialogues.append((
                    matched_face,
                    dialogue_entry['dialogue'],
                    dialogue_entry['character']
                ))

                # Remove the matched face to prevent double assignment
                if matched_face in identified_faces:
                    identified_faces.remove(matched_face)

        return matched_dialogues

    def create_speech_bubble(self, draw, x, y, width, height, text, radius=10, color=(255, 255, 255), pointer_direction="down"):
        """
        Draw a rounded rectangle speech bubble with text.

        Args:
            pointer_direction: Direction of the bubble pointer ("down", "left", "right")
        """
        draw.rounded_rectangle(
            [(x, y), (x + width, y + height)],
            radius=radius,
            fill=color,
            outline=(0, 0, 0)
        )

        # Pointer (triangle) for bubble based on direction
        if pointer_direction == "down":
            pointer_points = [
                (x + width//4, y + height),
                (x + width//4 + 10, y + height + 15),
                (x + width//4 + 20, y + height)
            ]
        elif pointer_direction == "left":
            pointer_points = [
                (x, y + height//2 - 10),
                (x - 15, y + height//2),
                (x, y + height//2 + 10)
            ]
        elif pointer_direction == "right":
            pointer_points = [
                (x + width, y + height//2 - 10),
                (x + width + 15, y + height//2),
                (x + width, y + height//2 + 10)
            ]

        draw.polygon(pointer_points, fill=color, outline=(0, 0, 0))

        # Text wrapping with better error handling
        text_width = width - 20
        try:
            avg_char_width = self.font.getlength('x')
        except:
            avg_char_width = 10  # Fallback value

        # Make sure chars_per_line is an integer to avoid errors in textwrap
        chars_per_line = max(1, int(text_width // avg_char_width))

        lines = textwrap.wrap(text, width=chars_per_line)

        try:
            line_height = self.font.getbbox('hg')[3] + 5
        except:
            line_height = 20  # Fallback value

        text_y = y + 10
        for line in lines:
            draw.text((x + 10, text_y), line, fill=(0, 0, 0), font=self.font)
            text_y += line_height

    def calculate_bubble_dimensions(self, dialogue, image_width, face_width):
        """Calculate appropriate bubble dimensions focusing on width rather than height."""
        # Get approximate text dimensions based on character count
        text_length = len(dialogue)

        try:
            avg_char_width = self.font.getlength('x')
        except:
            avg_char_width = 10  # Fallback

        try:
            line_height = self.font.getbbox('hg')[3] + 5
        except:
            line_height = 20  # Fallback

        # Calculate maximum width based on image constraints
        max_bubble_width = min(int(image_width * 0.75), 500)  # Limit to 75% of image width or 500px
        min_bubble_width = max(300, face_width * 2)  # At least 300px or twice face width

        # Start with a wide bubble
        initial_bubble_width = min(max_bubble_width, max(min_bubble_width, int(text_length * avg_char_width * 0.6)))

        # Calculate how many lines the text will take
        chars_per_line = max(1, int((initial_bubble_width - 20) // avg_char_width))
        lines = textwrap.wrap(dialogue, width=chars_per_line)
        num_lines = len(lines)

        # Now calculate the height needed
        bubble_height = max(80, num_lines * line_height + 30)

        # If height is too tall compared to width, increase width to reduce height
        if bubble_height > initial_bubble_width * 0.75 and num_lines > 3:
            # Make the bubble wider to reduce height
            adjusted_width = min(max_bubble_width, int(initial_bubble_width * 1.5))

            # Recalculate lines with new width
            chars_per_line = max(1, int((adjusted_width - 20) // avg_char_width))
            lines = textwrap.wrap(dialogue, width=chars_per_line)
            num_lines = len(lines)

            # Final height calculation
            bubble_height = max(80, num_lines * line_height + 30)
            return adjusted_width, bubble_height

        return initial_bubble_width, bubble_height

    def process_panel(self, panel_image_path, panel_dialogues, output_folder="output_panels"):
        """Process a single panel with its dialogues."""
        print(f"Processing {panel_image_path} with {len(panel_dialogues)} dialogues")

        img, rgb_img, faces = self.detect_faces(panel_image_path)

        if rgb_img is None:
            print(f"Failed to process {panel_image_path}")
            return None, None

        image_height, image_width = rgb_img.shape[:2]

        # Pass rgb_img to assign_dialogues_to_faces for character identification
        face_dialogues = self.assign_dialogues_to_faces(faces, panel_dialogues, image_width, image_height, rgb_img)

        pil_img = Image.fromarray(rgb_img)
        draw = ImageDraw.Draw(pil_img)

        for face, dialogue, character in face_dialogues:
            if not dialogue:
                continue

            x, y, width, height = face['box']
            is_virtual = face.get('virtual', False)
            is_top_left = face.get('top_left', False)

            # Calculate bubble dimensions prioritizing width over height
            bubble_width, bubble_height = self.calculate_bubble_dimensions(dialogue, image_width, width)

            # Determine bubble position and pointer direction
            pointer_direction = "down"

            if is_top_left:
                # Position in top left with no pointer
                bubble_x = 10
                bubble_y = y
                pointer_direction = None  # No pointer for top left bubbles
            elif is_virtual:
                # For other virtual faces, position bubble above the virtual face position
                bubble_x = max(10, min(image_width - bubble_width - 10, x - (bubble_width - width)//2))
                bubble_y = max(10, y - bubble_height - 20)
            else:
                # For detected faces, use normal positioning logic
                bubble_x = max(10, min(image_width - bubble_width - 10, x - (bubble_width - width)//2))
                bubble_y = max(10, y - bubble_height - 20)

                # If bubble would go off top of frame, reposition
                if bubble_y < 10:
                    bubble_y = max(10, y + height + 10)  # Position below face

                    # If that still doesn't work well, try sides
                    if bubble_y + bubble_height > image_height - 10:
                        if x > image_width // 2:
                            # Face is on right side, put bubble on left
                            bubble_x = max(10, x - bubble_width - 10)
                            bubble_y = max(10, min(image_height - bubble_height - 10, y))
                            pointer_direction = "right"
                        else:
                            # Face is on left side, put bubble on right
                            bubble_x = min(image_width - bubble_width - 10, x + width + 10)
                            bubble_y = max(10, min(image_height - bubble_height - 10, y))
                            pointer_direction = "left"

            # Use only the dialogue text without the character name
            dialogue_text = dialogue

            # Modified create_speech_bubble call to handle None pointer_direction
            if pointer_direction is None:
                # Create bubble without pointer
                draw.rounded_rectangle(
                    [(bubble_x, bubble_y), (bubble_x + bubble_width, bubble_y + bubble_height)],
                    radius=10,
                    fill=(255, 255, 255),
                    outline=(0, 0, 0)
                )

                # Add text directly
                text_width = bubble_width - 20
                try:
                    avg_char_width = self.font.getlength('x')
                except:
                    avg_char_width = 10

                chars_per_line = max(1, int(text_width // avg_char_width))
                lines = textwrap.wrap(dialogue_text, width=chars_per_line)
                text_y = bubble_y + 10

                try:
                    line_height = self.font.getbbox('hg')[3] + 5
                except:
                    line_height = 20

                for line in lines:
                    draw.text((bubble_x + 10, text_y), line, fill=(0, 0, 0), font=self.font)
                    text_y += line_height
            else:
                # Use normal speech bubble with pointer
                self.create_speech_bubble(
                    draw,
                    bubble_x,
                    bubble_y,
                    bubble_width,
                    bubble_height,
                    dialogue_text,
                    pointer_direction=pointer_direction
                )

            # REMOVED: Debug rectangles and character name labels are no longer drawn

        os.makedirs(output_folder, exist_ok=True)

        panel_number = os.path.basename(panel_image_path).split('_')[1]
        output_path = os.path.join(output_folder, f"panel_{panel_number}_with_dialogue.png")

        pil_img.save(output_path)
        print(f"Saved panel with dialogue bubbles to {output_path}")

        return pil_img, output_path

    def process_all_panels(self, dialogues_file, panels_folder, output_folder="output_panels"):
        """Process all panels with their respective dialogues."""
        with open(dialogues_file, 'r') as f:
            all_dialogues = json.load(f)

        panel_files = glob.glob(os.path.join(panels_folder, "panel_*_best.png"))
        panel_files.sort(key=lambda x: int(re.search(r'panel_(\d+)_best', x).group(1)))

        processed_panels = []

        for panel_file in panel_files:
            panel_match = re.search(r'panel_(\d+)_best', panel_file)
            if not panel_match:
                print(f"Skipping {panel_file} - cannot determine panel number")
                continue

            panel_number = panel_match.group(1)
            panel_key = f"panel_{panel_number}"

            if panel_key in all_dialogues:
                panel_dialogues = all_dialogues[panel_key]
                result = self.process_panel(panel_file, panel_dialogues, output_folder)
                if result[1] is not None:  # If processing was successful
                    processed_panels.append(result[1])
            else:
                print(f"No dialogues found for {panel_key}, skipping")

        return processed_panels

    def visualize_result(self, image_path):
        """Display a result image."""
        img = Image.open(image_path)
        plt.figure(figsize=(12, 8))
        plt.imshow(np.array(img))
        plt.axis('off')
        plt.title(os.path.basename(image_path))
        plt.show()

# Example usage
if __name__ == "__main__":
    mapper = DialogueMapper(use_opencv_fallback=True)  # Enable OpenCV fallback

    dialogues_file = "comic_dialogues.json"   # Path to JSON file
    panels_folder = "comic_panels"             # Folder containing panel images
    output_folder = "output_panels"

    processed_panels = mapper.process_all_panels(dialogues_file, panels_folder, output_folder)

    print(f"Successfully processed {len(processed_panels)} panels!")
    print("Output files:")
    for panel in processed_panels:
        print(f"- {panel}")

Loaded bold font: /usr/share/fonts/liberation/LiberationSans-Bold.ttf
Successfully loaded Haar cascade classifier
Successfully loaded DNN face detector
Processing comic_panels/panel_1_best.png with 1 dialogues
RetinaFace detection error: Exception encountered when calling MaxPooling2D.call().

Argument `padding` must be either 'valid' or 'same'. Received: padding=VALID

Arguments received by MaxPooling2D.call():
  • args=('<KerasTensor shape=(None, None, None, 64), dtype=float32, sparse=False, name=keras_tensor_5>',)
  • kwargs=<class 'inspect._empty'>
Using OpenCV face detection
DNN detected 3 faces
Total OpenCV detected 3 faces
Identified face as: Unknown
Identified face as: Unknown
Identified face as: Unknown
Saved panel with dialogue bubbles to output_panels/panel_1_with_dialogue.png
Processing comic_panels/panel_2_best.png with 1 dialogues
RetinaFace detection error: Exception encountered when calling MaxPooling2D.call().

Argument `padding` must be either 'valid' or 'same'. Recei

Text-to-Speech using Web Speech API (specifically speechSynthesis) :

In [ ]:
# Interactive Comic Viewer - All Panels Display with Click-to-Speak
# This code displays all comic panels and allows clicking on any panel to hear its dialogue

import json
import os
import base64
from IPython.display import display, HTML, Javascript
from google.colab import files
import ipywidgets as widgets
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.widgets import Button
import numpy as np
import glob
from io import BytesIO

class InteractiveComicViewer:
    def __init__(self):
        self.comic_images = {}
        self.dialogue_data = {}
        self.image_folder = "/content/output_panels"
        self.json_file = "/content/comic_dialogues.json"

        print("🎭 Interactive Comic Viewer - All Panels Display")
        print("=" * 50)

    def load_existing_files(self):
        """Load existing comic panels and dialogue data"""
        print("📁 Loading existing files...")
        print("-" * 30)

        # Load comic images from output_panels folder
        if os.path.exists(self.image_folder):
            image_files = glob.glob(os.path.join(self.image_folder, "panel_*_with_dialogue.png"))

            for image_path in image_files:
                filename = os.path.basename(image_path)
                # Extract panel number from filename like "panel_1_with_dialogue.png"
                panel_num = self.extract_panel_number(filename)

                self.comic_images[panel_num] = {
                    'filename': filename,
                    'path': image_path
                }
                print(f"✅ Found panel {panel_num}: {filename}")

            print(f"📸 Loaded {len(self.comic_images)} comic panels")
        else:
            print(f"❌ Image folder not found: {self.image_folder}")
            return False

        # Load dialogue data
        if os.path.exists(self.json_file):
            try:
                with open(self.json_file, 'r', encoding='utf-8') as f:
                    self.dialogue_data = json.load(f)
                print(f"✅ Loaded dialogue data from {self.json_file}")
                print(f"📊 Found dialogues for {len(self.dialogue_data)} panels")
            except Exception as e:
                print(f"❌ Error loading JSON: {e}")
                return False
        else:
            print(f"❌ JSON file not found: {self.json_file}")
            return False

        return len(self.comic_images) > 0 and len(self.dialogue_data) > 0

    def extract_panel_number(self, filename):
        """Extract panel number from filename like 'panel_1_with_dialogue.png'"""
        import re
        # Look for pattern like "panel_X_" where X is the number
        match = re.search(r'panel_(\d+)_', filename)
        if match:
            return match.group(1)
        else:
            # Fallback to any number in filename
            numbers = re.findall(r'\d+', filename)
            return numbers[0] if numbers else "1"

    def image_to_base64(self, image_path):
        """Convert image to base64 for HTML display"""
        try:
            with open(image_path, "rb") as img_file:
                img_data = img_file.read()
                img_base64 = base64.b64encode(img_data).decode('utf-8')
                return f"data:image/png;base64,{img_base64}"
        except Exception as e:
            print(f"Error converting image {image_path}: {e}")
            return None

    def create_all_panels_viewer(self):
        """Create HTML viewer showing all panels with click-to-speak functionality"""
        if not self.comic_images or not self.dialogue_data:
            print("❌ No comic images or dialogue data found!")
            return

        print("\n🎮 Displaying All Comic Panels")
        print("-" * 30)

        # Sort panels by number
        sorted_panels = sorted(self.comic_images.keys(),
                             key=lambda x: int(x) if x.isdigit() else float('inf'))

        # Generate HTML for all panels
        panels_html = ""
        for panel_num in sorted_panels:
            panel_info = self.comic_images[panel_num]
            image_base64 = self.image_to_base64(panel_info['path'])

            if image_base64:
                panels_html += f"""
                <div class="panel-container" style="margin: 20px 0; text-align: center;">
                    <h3 style="color: #333; margin-bottom: 10px;">📖 Panel {panel_num}</h3>
                    <img id="panel_{panel_num}"
                         src="{image_base64}"
                         alt="Panel {panel_num}"
                         onclick="speakPanelDialogue('{panel_num}')"
                         style="max-width: 800px; width: 100%; height: auto;
                                border: 3px solid #ddd; border-radius: 10px;
                                cursor: pointer; transition: all 0.3s ease;
                                box-shadow: 0 4px 8px rgba(0,0,0,0.1);"
                         onmouseover="this.style.borderColor='#4CAF50'; this.style.boxShadow='0 6px 12px rgba(0,0,0,0.2)';"
                         onmouseout="this.style.borderColor='#ddd'; this.style.boxShadow='0 4px 8px rgba(0,0,0,0.1)';">
                    <p style="color: #666; font-style: italic; margin-top: 10px;">
                        🔊 Click on the panel to hear the dialogue
                    </p>
                </div>
                """

        # Create complete HTML with JavaScript
        html_code = f"""
        <div style="font-family: Arial, sans-serif; max-width: 1200px; margin: 0 auto; padding: 20px;">
            <div style="text-align: center; margin-bottom: 30px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                        color: white; padding: 20px; border-radius: 15px;">
                <h1 style="margin: 0; font-size: 28px;">🎭 Interactive Comic Viewer</h1>
                <p style="margin: 10px 0 0 0; font-size: 16px;">Click on any panel to hear its dialogue!</p>
            </div>

            <div style="text-align: center; margin: 20px 0;">
                <button onclick="stopAllSpeech()"
                        style="padding: 12px 24px; background: #f44336; color: white;
                               border: none; border-radius: 8px; cursor: pointer;
                               font-size: 16px; margin: 0 10px;">
                    ⏹️ Stop All Speech
                </button>
                <button onclick="showDialogueInfo()"
                        style="padding: 12px 24px; background: #2196F3; color: white;
                               border: none; border-radius: 8px; cursor: pointer;
                               font-size: 16px; margin: 0 10px;">
                    📝 Show All Dialogues
                </button>
            </div>

            <div id="dialogueInfo" style="background: #f9f9f9; padding: 20px; border-radius: 10px;
                                          margin: 20px 0; display: none;">
                <h3>📋 Dialogue Information</h3>
                <div id="dialogueContent"></div>
            </div>

            <div id="panelsContainer">
                {panels_html}
            </div>
        </div>

        <script>
            // Dialogue data from JSON
            const dialogueData = {json.dumps(self.dialogue_data, indent=2)};

            // Function to speak panel dialogue
            function speakPanelDialogue(panelNum) {{
                console.log('Clicked on panel:', panelNum);

                // Stop any ongoing speech
                if (window.speechSynthesis) {{
                    window.speechSynthesis.cancel();
                }}

                const panelKey = 'panel_' + panelNum;

                if (dialogueData[panelKey]) {{
                    const dialogues = dialogueData[panelKey];
                    console.log('Found dialogues:', dialogues);

                    // Visual feedback
                    const panelImg = document.getElementById('panel_' + panelNum);
                    if (panelImg) {{
                        panelImg.style.borderColor = '#FF5722';
                        panelImg.style.transform = 'scale(1.02)';
                        setTimeout(() => {{
                            panelImg.style.borderColor = '#4CAF50';
                            panelImg.style.transform = 'scale(1)';
                        }}, 200);
                    }}

                    // Speak each dialogue with proper timing
                    let totalDelay = 0;
                    dialogues.forEach((dialogueObj, index) => {{
                        setTimeout(() => {{
                            const character = dialogueObj.character || 'Unknown';
                            const dialogue = dialogueObj.dialogue || '';
                            const textToSpeak = character + ' says: ' + dialogue;

                            console.log('Speaking:', textToSpeak);
                            speakText(textToSpeak);
                        }}, totalDelay);

                        // Calculate delay for next dialogue (estimate speaking time)
                        const estimatedDuration = Math.max(2000, (dialogueObj.dialogue || '').split(' ').length * 400);
                        totalDelay += estimatedDuration;
                    }});

                }} else {{
                    console.log('No dialogue found for panel:', panelNum);
                    speakText('No dialogue found for panel ' + panelNum);
                }}
            }}

            // Text-to-speech function
            function speakText(text) {{
                if ('speechSynthesis' in window) {{
                    const utterance = new SpeechSynthesisUtterance(text);

                    // Configure voice settings
                    utterance.rate = 0.8;
                    utterance.pitch = 1.0;
                    utterance.volume = 0.9;

                    // Try to use English voice
                    const voices = speechSynthesis.getVoices();
                    const englishVoice = voices.find(voice => voice.lang.startsWith('en'));
                    if (englishVoice) {{
                        utterance.voice = englishVoice;
                    }}

                    // Event listeners
                    utterance.onstart = () => console.log('Started speaking:', text);
                    utterance.onend = () => console.log('Finished speaking');
                    utterance.onerror = (e) => console.log('Speech error:', e);

                    speechSynthesis.speak(utterance);
                }} else {{
                    alert('Speech synthesis not supported in your browser');
                }}
            }}

            // Stop all speech
            function stopAllSpeech() {{
                if (window.speechSynthesis) {{
                    window.speechSynthesis.cancel();
                    console.log('All speech stopped');
                }}
            }}

            // Show dialogue information
            function showDialogueInfo() {{
                const infoDiv = document.getElementById('dialogueInfo');
                const contentDiv = document.getElementById('dialogueContent');

                if (infoDiv.style.display === 'none' || !infoDiv.style.display) {{
                    // Show dialogue info
                    let content = '';
                    Object.keys(dialogueData).sort((a, b) => {{
                        const numA = parseInt(a.replace('panel_', ''));
                        const numB = parseInt(b.replace('panel_', ''));
                        return numA - numB;
                    }}).forEach(panelKey => {{
                        const panelNum = panelKey.replace('panel_', '');
                        const dialogues = dialogueData[panelKey];

                        content += `<div style="margin: 15px 0; padding: 15px; background: white; border-radius: 8px; border-left: 4px solid #4CAF50;">`;
                        content += `<h4 style="margin: 0 0 10px 0; color: #333;">📖 Panel ${{panelNum}}</h4>`;

                        dialogues.forEach((dialogue, index) => {{
                            content += `<div style="margin: 8px 0; padding: 8px; background: #f0f8ff; border-radius: 4px;">`;
                            content += `<strong style="color: #2196F3;">${{dialogue.character}}:</strong> `;
                            content += `<span style="color: #333;">${{dialogue.dialogue}}</span>`;
                            content += `</div>`;
                        }});

                        content += `</div>`;
                    }});

                    contentDiv.innerHTML = content;
                    infoDiv.style.display = 'block';
                }} else {{
                    // Hide dialogue info
                    infoDiv.style.display = 'none';
                }}
            }}

            // Initialize voices (some browsers need this)
            if ('speechSynthesis' in window) {{
                speechSynthesis.onvoiceschanged = function() {{
                    console.log('Voices loaded:', speechSynthesis.getVoices().length);
                }};
            }}
        </script>
        """

        display(HTML(html_code))

        # Display summary
        print(f"\n✅ Displayed {len(sorted_panels)} comic panels")
        print("🖱️ Click on any panel image to hear its dialogue!")
        print("🔊 Use the control buttons for additional features")

    def show_file_info(self):
        """Display information about loaded files"""
        print("📋 File Information:")
        print("=" * 30)

        print(f"📁 Image folder: {self.image_folder}")
        print(f"📄 JSON file: {self.json_file}")

        print(f"\n📸 Comic Images ({len(self.comic_images)}):")
        for panel_num, info in sorted(self.comic_images.items(), key=lambda x: int(x[0]) if x[0].isdigit() else float('inf')):
            print(f"  Panel {panel_num}: {info['filename']}")

        print(f"\n💬 Dialogue Data:")
        for panel_key in sorted(self.dialogue_data.keys()):
            panel_num = panel_key.replace('panel_', '')
            dialogue_count = len(self.dialogue_data[panel_key])
            print(f"  {panel_key}: {dialogue_count} dialogue(s)")

    def run(self):
        """Main method to run the interactive comic viewer"""
        print("🚀 Starting Interactive Comic Viewer...")
        print("Loading files from your specified locations:\n")

        # Load existing files
        if self.load_existing_files():
            self.show_file_info()
            print("\n" + "="*50)
            self.create_all_panels_viewer()
        else:
            print("❌ Could not load files. Please check that:")
            print(f"   • Folder exists: {self.image_folder}")
            print(f"   • JSON file exists: {self.json_file}")
            print(f"   • Image files follow pattern: panel_X_with_dialogue.png")

# Additional utility functions

def quick_test():
    """Quick test to check if files exist"""
    image_folder = "/content/output_panels"
    json_file = "/content/comic_dialogues.json"

    print("🔍 Quick File Check:")
    print("-" * 20)

    # Check image folder
    if os.path.exists(image_folder):
        image_files = glob.glob(os.path.join(image_folder, "panel_*_with_dialogue.png"))
        print(f"✅ Image folder exists: {len(image_files)} panel files found")
        for f in sorted(image_files)[:5]:  # Show first 5
            print(f"   • {os.path.basename(f)}")
        if len(image_files) > 5:
            print(f"   • ... and {len(image_files)-5} more")
    else:
        print(f"❌ Image folder not found: {image_folder}")

    # Check JSON file
    if os.path.exists(json_file):
        try:
            with open(json_file, 'r') as f:
                data = json.load(f)
            print(f"✅ JSON file exists with {len(data)} panels")

            # Show sample dialogue
            sample_key = list(data.keys())[0] if data else None
            if sample_key:
                sample_dialogues = data[sample_key]
                print(f"   Sample from {sample_key}:")
                for dialogue in sample_dialogues[:2]:  # Show first 2
                    char = dialogue.get('character', 'Unknown')
                    text = dialogue.get('dialogue', '')[:50] + '...' if len(dialogue.get('dialogue', '')) > 50 else dialogue.get('dialogue', '')
                    print(f"     {char}: {text}")
        except Exception as e:
            print(f"❌ JSON file error: {e}")
    else:
        print(f"❌ JSON file not found: {json_file}")

def main():
    """Main function to run the interactive comic viewer"""

    print("""
    🎭 INTERACTIVE COMIC VIEWER - ALL PANELS DISPLAY
    ===============================================

    This version displays ALL comic panels at once:
    📁 Images: /content/output_panels/panel_X_with_dialogue.png
    📄 Dialogues: /content/comic_dialogues.json

    Features:
    ✅ Shows all panels in one view
    ✅ Click on any panel to hear its dialogue
    ✅ Visual feedback when clicking5
    ✅ Speech controls and dialogue info
    ✅ Automatic file loading
    """)

    # Quick file check first
    quick_test()

    print("\n" + "="*50)

    # Create and run the viewer
    viewer = InteractiveComicViewer()
    viewer.run()

# Run the application
if __name__ == "__main__":
    main()

# Uncomment the line below to just run the quick test
quick_test()

Text-to-Speech using OCR :

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract Pillow
# Install the Python wrapper
!pip install pytesseract --quiet

# Install Tesseract OCR (the engine)
!apt-get install -y tesseract-ocr


In [ ]:
import cv2
import numpy as np
import os
from PIL import Image, ImageEnhance, ImageFilter
import pytesseract
import re
from collections import defaultdict

# Configure pytesseract path if needed (uncomment and modify if required)
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Folder containing the comic panels
input_folder = "/content/output_panels"  # Change this to your folder path
output_folder = "ocr_results"  # Folder to save OCR results

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

def detect_speech_bubbles(image):
    """
    Improved speech bubble detection using multiple techniques
    """
    try:
        # Create a copy for processing
        work_img = image.copy()

        # Apply morphological operations to connect text regions
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20, 20))
        closed = cv2.morphologyEx(255 - work_img, cv2.MORPH_CLOSE, kernel)

        # Find contours
        contours, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Filter contours for speech bubble characteristics
        speech_regions = []
        height, width = image.shape
        min_area = (height * width) * 0.005  # At least 0.5% of image
        max_area = (height * width) * 0.4    # At most 40% of image

        for contour in contours:
            area = cv2.contourArea(contour)
            if min_area < area < max_area:
                x, y, w, h = cv2.boundingRect(contour)

                # Check aspect ratio and minimum size
                aspect_ratio = w / h if h > 0 else 0
                if 0.2 < aspect_ratio < 6.0 and w > 30 and h > 20:
                    # Add some padding around detected region
                    padding = 10
                    x = max(0, x - padding)
                    y = max(0, y - padding)
                    w = min(width - x, w + 2 * padding)
                    h = min(height - y, h + 2 * padding)

                    speech_regions.append((x, y, w, h))

        return speech_regions

    except Exception as e:
        print(f"Error in speech bubble detection: {str(e)}")
        return []

def preprocess_for_text_detection(image_path):
    """
    Optimized preprocessing specifically for comic text detection
    """
    try:
        # Read image
        img = cv2.imread(image_path)
        if img is None:
            raise ValueError(f"Could not read image: {image_path}")

        # Convert to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Apply bilateral filter to reduce noise while preserving edges
        filtered = cv2.bilateralFilter(gray, 9, 75, 75)

        # Enhance contrast using CLAHE (Contrast Limited Adaptive Histogram Equalization)
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        enhanced = clahe.apply(filtered)

        # Apply threshold to get binary image
        # Use Otsu's method for automatic threshold selection
        _, binary = cv2.threshold(enhanced, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        return binary, gray

    except Exception as e:
        print(f"Error in preprocessing: {str(e)}")
        return None, None

def preprocess_text_region(region_img, scale_factor=2):
    """
    Specialized preprocessing for individual text regions
    """
    try:
        # Resize for better OCR accuracy
        height, width = region_img.shape
        new_width = int(width * scale_factor)
        new_height = int(height * scale_factor)
        resized = cv2.resize(region_img, (new_width, new_height), interpolation=cv2.INTER_CUBIC)

        # Convert to PIL for enhancement
        pil_img = Image.fromarray(resized)

        # Enhance contrast
        enhancer = ImageEnhance.Contrast(pil_img)
        pil_img = enhancer.enhance(1.2)

        # Enhance sharpness slightly
        enhancer = ImageEnhance.Sharpness(pil_img)
        pil_img = enhancer.enhance(1.1)

        # Apply slight blur to smooth out artifacts
        pil_img = pil_img.filter(ImageFilter.GaussianBlur(radius=0.5))

        return pil_img, scale_factor

    except Exception as e:
        print(f"Error in text region preprocessing: {str(e)}")
        return None, scale_factor

def clean_comic_text(raw_text):
    """
    Improved text cleaning specifically for comic dialogue
    """
    if not raw_text or not raw_text.strip():
        return ""

    try:
        # Remove extra whitespace and newlines
        text = ' '.join(raw_text.split())

        # Common OCR corrections for comics
        corrections = {
            # Letter substitutions
            '0': 'O', '1': 'I', '5': 'S', '8': 'B', '6': 'G',
            'rn': 'm', 'cl': 'd', 'vv': 'w', 'ii': 'll',
            'rnm': 'm', 'nm': 'm', 'nn': 'n',
            # Remove common OCR artifacts
            '|': 'l', '!': 'l', '£': 'E', '@': 'a',
            # Fix punctuation
            ' ,': ',', ' .': '.', ' !': '!', ' ?': '?',
            ',,': ',', '..': '.', '!!': '!', '??': '?'
        }

        # Apply corrections
        for wrong, right in corrections.items():
            text = text.replace(wrong, right)

        # Remove sequences of repeated characters that are likely noise
        text = re.sub(r'(.)\1{4,}', r'\1', text)  # Remove 5+ repeated chars

        # Remove standalone single characters that are likely noise
        words = text.split()
        filtered_words = []

        for word in words:
            # Keep word if it's longer than 1 char, or if it's a common single letter word
            if len(word) > 1 or word.upper() in ['I', 'A']:
                # Remove words that are mostly special characters
                if sum(c.isalnum() for c in word) >= len(word) * 0.7:
                    filtered_words.append(word)

        result = ' '.join(filtered_words)

        # Final cleanup
        result = re.sub(r'\s+', ' ', result).strip()

        # Remove if result is too short or contains too many non-alphabetic characters
        if len(result) < 3:
            return ""

        alpha_ratio = sum(c.isalpha() for c in result) / len(result) if result else 0
        if alpha_ratio < 0.5:  # Less than 50% alphabetic characters
            return ""

        return result

    except Exception as e:
        print(f"Error cleaning text: {str(e)}")
        return raw_text.strip()

def extract_text_from_region(region_img):
    """
    Extract text from a specific image region using optimized OCR
    """
    try:
        # Preprocess the region
        processed_img, scale_factor = preprocess_text_region(region_img)
        if processed_img is None:
            return []

        # OCR configuration optimized for comic text
        custom_config = r'--oem 3 --psm 6 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789.,!?:;-\' '

        # Extract text
        text = pytesseract.image_to_string(processed_img, config=custom_config)

        # Clean the extracted text
        cleaned_text = clean_comic_text(text)

        if cleaned_text:
            return [cleaned_text]

        # Fallback: try with different PSM mode
        fallback_config = r'--oem 3 --psm 8 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789.,!?:;-\' '
        text = pytesseract.image_to_string(processed_img, config=fallback_config)
        cleaned_text = clean_comic_text(text)

        return [cleaned_text] if cleaned_text else []

    except Exception as e:
        print(f"Error extracting text from region: {str(e)}")
        return []

def perform_enhanced_ocr(image_path):
    """
    Main OCR function with improved speech bubble detection
    """
    try:
        print(f"    Processing image: {os.path.basename(image_path)}")

        # Preprocess image for text detection
        binary_img, gray_img = preprocess_for_text_detection(image_path)
        if binary_img is None:
            return []

        # Save preprocessed image for debugging
        debug_path = os.path.join(output_folder, f"debug_{os.path.basename(image_path)}")
        cv2.imwrite(debug_path, binary_img)

        # Detect speech bubble regions
        speech_regions = detect_speech_bubbles(binary_img)
        print(f"    Found {len(speech_regions)} potential speech regions")

        all_text = []

        if speech_regions:
            # Process each detected region
            for i, (x, y, w, h) in enumerate(speech_regions):
                print(f"    Processing region {i+1}/{len(speech_regions)}")

                # Extract region from grayscale image
                region = gray_img[y:y+h, x:x+w]

                # Extract text from this region
                region_text = extract_text_from_region(region)

                for text in region_text:
                    if text and len(text) > 2:
                        all_text.append(text)
                        print(f"      Found: \"{text}\"")

        # If no regions detected or no text found, try processing entire image
        if not all_text:
            print("    No text found in regions, trying full image...")
            full_text = extract_text_from_region(gray_img)
            all_text.extend(full_text)

        return all_text

    except Exception as e:
        print(f"    Error in OCR processing: {str(e)}")
        return []

def main():
    """
    Main processing function
    """
    # Get all panel images from the folder
    try:
        all_files = os.listdir(input_folder)
        panel_files = [f for f in all_files if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        # Sort the panel files numerically if they contain numbers
        def extract_number(filename):
            numbers = re.findall(r'\d+', filename)
            return int(numbers[0]) if numbers else float('inf')

        panel_files.sort(key=extract_number)

    except Exception as e:
        print(f"Error accessing input folder '{input_folder}': {str(e)}")
        return

    print(f"Found {len(panel_files)} panel images to process")
    print("Processing with enhanced OCR for comic speech bubbles...\n")

    # Process each panel
    all_results = {}
    successful_panels = 0

    for panel_file in panel_files:
        img_path = os.path.join(input_folder, panel_file)

        # Extract panel number from filename
        numbers = re.findall(r'\d+', panel_file)
        panel_number = numbers[0] if numbers else panel_file.replace('.png', '').replace('.jpg', '').replace('.jpeg', '')

        print(f"Processing Panel {panel_number} ({panel_file})...")

        try:
            # Perform enhanced OCR
            extracted_text = perform_enhanced_ocr(img_path)

            # Filter results
            clean_text = []
            for text in extracted_text:
                if len(text) > 3 and not text.isdigit():
                    clean_text.append(text)

            all_results[panel_number] = clean_text

            # Print results
            if clean_text:
                print(f"  ✅ Successfully extracted {len(clean_text)} dialogue lines:")
                for i, text in enumerate(clean_text, 1):
                    print(f"     {i}. \"{text}\"")
                successful_panels += 1
            else:
                print(f"  ⚠️  No clear dialogue detected in this panel")

            # Save individual panel results
            result_path = os.path.join(output_folder, f"panel_{panel_number}_dialogue.txt")
            with open(result_path, "w", encoding='utf-8') as f:
                f.write(f"Panel {panel_number} - Extracted Dialogue\n")
                f.write("=" * 40 + "\n\n")
                f.write(f"Source Image: {panel_file}\n\n")

                if clean_text:
                    for i, text in enumerate(clean_text, 1):
                        f.write(f"Line {i}: {text}\n")
                    f.write(f"\nTotal lines extracted: {len(clean_text)}\n")
                else:
                    f.write("No dialogue detected in this panel.\n")

            print(f"  💾 Results saved to {result_path}\n")

        except Exception as e:
            print(f"  ❌ Error processing panel {panel_number}: {str(e)}\n")
            all_results[panel_number] = []

    # Save combined results
    combined_path = os.path.join(output_folder, "all_panels_dialogue.txt")
    try:
        with open(combined_path, "w", encoding='utf-8') as f:
            f.write("COMPLETE COMIC DIALOGUE EXTRACTION\n")
            f.write("=" * 50 + "\n\n")
            f.write(f"Input Folder: {input_folder}\n")
            f.write(f"Total Images Processed: {len(panel_files)}\n\n")

            total_lines = 0
            panels_with_dialogue = 0

            # Sort results by panel number
            sorted_panels = sorted(all_results.keys(), key=lambda x: int(x) if x.isdigit() else float('inf'))

            for panel_num in sorted_panels:
                text_lines = all_results[panel_num]
                f.write(f"PANEL {panel_num}\n")
                f.write("-" * 20 + "\n")

                if text_lines:
                    for i, text in enumerate(text_lines, 1):
                        f.write(f"  {text}\n")
                        total_lines += 1
                    f.write(f"  [{len(text_lines)} lines]\n\n")
                    panels_with_dialogue += 1
                else:
                    f.write("  [No dialogue detected]\n\n")

            f.write("=" * 50 + "\n")
            f.write("EXTRACTION SUMMARY\n")
            f.write("=" * 50 + "\n")
            f.write(f"Total Panels Processed: {len(panel_files)}\n")
            f.write(f"Panels with Dialogue: {panels_with_dialogue}\n")
            f.write(f"Total Dialogue Lines: {total_lines}\n")
            f.write(f"Success Rate: {(panels_with_dialogue/len(panel_files)*100):.1f}%\n")

    except Exception as e:
        print(f"Error saving combined results: {str(e)}")

    print("=" * 60)
    print("🎉 PROCESSING COMPLETE!")
    print(f"📊 Results Summary:")
    print(f"   • Processed: {len(panel_files)} panels")
    print(f"   • Successful extractions: {successful_panels} panels")
    print(f"   • Total dialogue lines: {sum(len(text) for text in all_results.values())}")
    print(f"   • Success rate: {(successful_panels/len(panel_files)*100):.1f}%")
    print(f"   • Individual results: {output_folder}/panel_X_dialogue.txt")
    print(f"   • Combined results: {combined_path}")
    print(f"   • Debug images: {output_folder}/debug_*.png")
    print("=" * 60)

if __name__ == "__main__":
    main()

In [ ]:
import os
import glob
from PIL import Image
from IPython.display import HTML, display
import pytesseract
import cv2
import base64
from io import BytesIO

# === Step 1: Specify folder path ===
folder_path = "/content/output_panels"  # Update this to your image folder in Colab

# === Step 2: Get all image file paths from the folder ===
image_extensions = ["*.png", "*.jpg", "*.jpeg", "*.bmp"]
image_paths = []
for ext in image_extensions:
    image_paths.extend(glob.glob(os.path.join(folder_path, ext)))

# === Step 3: Loop through each image ===
img_display_width = 500  # Desired display width

for img_path in image_paths:
    # Load original color image for display
    image = Image.open(img_path)
    img_width, img_height = image.size

    # Load with OpenCV for OCR processing
    img_cv = cv2.imread(img_path)
    gray = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)

    # Resize for better OCR
    resized = cv2.resize(gray, (img_width * 2, img_height * 2), interpolation=cv2.INTER_LINEAR)

    # OCR processing with full data output
    ocr_data = pytesseract.image_to_data(Image.fromarray(resized), output_type=pytesseract.Output.DICT)

    # === Group words into speech bubbles using block_num ===
    blocks = {}
    for i in range(len(ocr_data['text'])):
        text = ocr_data['text'][i].strip()
        if text:
            block_num = ocr_data['block_num'][i]
            if block_num not in blocks:
                blocks[block_num] = {
                    'text': [],
                    'left': [],
                    'top': [],
                    'right': [],
                    'bottom': []
                }
            x, y, w, h = ocr_data['left'][i], ocr_data['top'][i], ocr_data['width'][i], ocr_data['height'][i]
            blocks[block_num]['text'].append(text)
            blocks[block_num]['left'].append(x)
            blocks[block_num]['top'].append(y)
            blocks[block_num]['right'].append(x + w)
            blocks[block_num]['bottom'].append(y + h)

    # Create list of bounding boxes for display
    bubble_boxes = []
    for block in blocks.values():
        full_text = " ".join(block['text'])
        x1, y1 = min(block['left']), min(block['top'])
        x2, y2 = max(block['right']), max(block['bottom'])
        bubble_boxes.append({
            "text": full_text,
            "x": x1 / 2,
            "y": y1 / 2,
            "w": (x2 - x1) / 2,
            "h": (y2 - y1) / 2
        })

    # === Scale for display ===
    scale = img_display_width / img_width
    boxes_html = ""
    for d in bubble_boxes:
        x = int(d["x"] * scale)
        y = int(d["y"] * scale)
        w = int(d["w"] * scale)
        h = int(d["h"] * scale)
        text = d["text"].replace("`", "'").replace('"', '&quot;')
        boxes_html += f"""
        <div onclick="speakText(`{text}`)" style="
          position: absolute;
          top: {y}px;
          left: {x}px;
          width: {w}px;
          height: {h}px;
          cursor: pointer;
          background-color: rgba(255,255,255,0);
          border: 1px solid rgba(0,0,255,0.3);
        " title="{text}"></div>
        """

    # === Convert image to base64 for HTML display ===
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue()).decode()
    img_src = f"data:image/png;base64,{img_str}"

    # === Final HTML for Colab ===
    html_code = f"""
    <div style="position: relative; width: {img_display_width}px; margin-bottom: 30px;">
      <img src="{img_src}" width="{img_display_width}px">
      {boxes_html}
    </div>
    <script>
      function speakText(text) {{
        const utterance = new SpeechSynthesisUtterance(text);
        utterance.lang = 'en-US';
        speechSynthesis.cancel();
        speechSynthesis.speak(utterance);
      }}
    </script>
    """
    display(HTML(html_code))


Arrange The panels in order :

In [ ]:
from PIL import Image
import os
import glob
import re

# === Folder where your panels are stored ===
folder_path = "/content/output_panels"

# === Get all panel images and sort them numerically ===
image_paths = glob.glob(os.path.join(folder_path, "panel_*_with_dialogue.png"))

def extract_panel_number(filename):
    """Extract panel number from filename like 'panel_1_with_dialogue.png'"""
    match = re.search(r'panel_(\d+)_', filename)
    if match:
        return int(match.group(1))
    return 0  # fallback for files that don't match pattern

# Sort by panel number (not alphabetically)
image_paths.sort(key=lambda x: extract_panel_number(os.path.basename(x)))

print(f"Found {len(image_paths)} panels:")
for path in image_paths:
    panel_num = extract_panel_number(os.path.basename(path))
    print(f"  Panel {panel_num}: {os.path.basename(path)}")

# === Set consistent size for all panels (e.g., 512x512) ===
panel_width = 512
panel_height = 512
panels_per_row = 3
panel_margin = 10  # spacing between panels

# === Resize and store panels ===
resized_panels = []
for path in image_paths:
    img = Image.open(path).convert("RGB")
    img = img.resize((panel_width, panel_height), Image.LANCZOS)
    resized_panels.append(img)

# === Calculate layout ===
total_panels = len(resized_panels)
if total_panels == 0:
    print("No panels found! Check if files exist in the specified folder.")
    exit()

rows = (total_panels + panels_per_row - 1) // panels_per_row
canvas_width = panels_per_row * (panel_width + panel_margin) - panel_margin
canvas_height = rows * (panel_height + panel_margin) - panel_margin

print(f"\nCreating comic page with {total_panels} panels:")
print(f"  Layout: {rows} rows × {panels_per_row} columns")
print(f"  Canvas size: {canvas_width}×{canvas_height} pixels")

# === Create blank canvas ===
comic_canvas = Image.new("RGB", (canvas_width, canvas_height), "white")

# === Paste each panel on the canvas ===
for idx, panel in enumerate(resized_panels):
    row = idx // panels_per_row
    col = idx % panels_per_row
    x = col * (panel_width + panel_margin)
    y = row * (panel_height + panel_margin)
    comic_canvas.paste(panel, (x, y))

    # Get panel number for logging
    original_path = image_paths[idx]
    panel_num = extract_panel_number(os.path.basename(original_path))
    print(f"  Placed Panel {panel_num} at position ({row+1}, {col+1})")

# === Save the result ===
output_path = "/content/comic_page_ordered.jpg"
comic_canvas.save(output_path)
print(f"\n✅ Comic page saved to: {output_path}")

# === Display the result (if in Jupyter/Colab) ===
try:
    comic_canvas.show()
except:
    print("Display not available in this environment")

# === Optional: Save as PNG for better quality ===
png_output_path = "/content/comic_page_ordered.png"
comic_canvas.save(png_output_path)
print(f"✅ High-quality version saved to: {png_output_path}")

Found 13 panels:
  Panel 1: panel_1_with_dialogue.png
  Panel 2: panel_2_with_dialogue.png
  Panel 3: panel_3_with_dialogue.png
  Panel 4: panel_4_with_dialogue.png
  Panel 5: panel_5_with_dialogue.png
  Panel 6: panel_6_with_dialogue.png
  Panel 7: panel_7_with_dialogue.png
  Panel 8: panel_8_with_dialogue.png
  Panel 9: panel_9_with_dialogue.png
  Panel 10: panel_10_with_dialogue.png
  Panel 11: panel_11_with_dialogue.png
  Panel 12: panel_12_with_dialogue.png
  Panel 13: panel_13_with_dialogue.png

Creating comic page with 13 panels:
  Layout: 5 rows × 3 columns
  Canvas size: 1556×2600 pixels
  Placed Panel 1 at position (1, 1)
  Placed Panel 2 at position (1, 2)
  Placed Panel 3 at position (1, 3)
  Placed Panel 4 at position (2, 1)
  Placed Panel 5 at position (2, 2)
  Placed Panel 6 at position (2, 3)
  Placed Panel 7 at position (3, 1)
  Placed Panel 8 at position (3, 2)
  Placed Panel 9 at position (3, 3)
  Placed Panel 10 at position (4, 1)
  Placed Panel 11 at position (4, 2